# Répertoire des bilans sociaux de grandes entreprises - focus sur les salariés en situation de handicap

Objectif : extraire et mettre en forme les données de bilan social des entreprises autre qu'EDF liées aux salariés et à ceux en situation d'handicap 

Méthodologie : explorer les données mises en forme d'EDF fournies puis extraire et mettre en forme de la même manière des données d'autres entreprises

## TODO

|N° Mission|N° Tâche|Statut |Description tâche|
|-|-|-|-|
|2|1|Done                   |Téléchargement et exploration sommaire des données EDF|
|-|-|-|-|
|2|2|Done                   |Etablissement de l'environnement de travail|
|-|-|-|-|
|2|3|Done                   |Choix des librairies / outils et premiers tests|
|2|3.1|Done                 |Test de docling pour extraire les tableaux de données des pdf|
|2|3.2|Done                 |Test de LangChain pour extraire les informations des csv|
|-|-|-|-|
|2|4|Done                   |Téléchargement et exploration sommaire des données d'autres entreprises|
|2|4.1|Done                 |Liste d'autres entreprises et liens vers leurs données|
|2|4.2|Done                 |Téléchargement des données (fichiers pdf/csv)|
|-|-|-|-|
|2|5|A faire                |Automatisation du téléchargement des données|
|-|-|-|-|
|2|6|Done                   |Evaluer la possibilité d'utiliser un outils d'extraction depuis pdf pour extraire les données|
|2|6.1|Done                 |Extraction des tableaux des fichiers pdf en csv|
|-|-|-|-|
|2|7|Done                   |Evaluer la possibilité d'utiliser un LLM pour extraire les données|
|2|7.1|Done                 |Installation et set-up de LangChain et d'un premier modèle|
|2|7.2|Done                 |Test du modèle de Ollama Qwen2.5|
|2|7.3|Done                 |Modification du prompt pour obtenir les informations recherchées depuis les fichiers csv|
|2|7.4|Done                 |Evaluation du prompt pour tous les fichiers d'une même entreprise - Exemple d'Auchan|
|2|7.5|Done                 |Evaluation du prompt pour tous les fichiers de toutes les entreprises|
|2|7.6|Done                 |Optimisation du prompt pour tous les fichiers de toutes les entreprises|
|-|-|-|-|
|2|8|Done                   |Nettoyage des données de sortie du prompt|
|2|8.1|Done                 |Obtention d'un dataframe avec nombre, % et années|
|2|8.2|Done                 |Transformation du dataframe (type, valeurs, disposition) pour correspondre aux données d'EDF|
|-|-|-|-|
|2|9|En cours                |Implémenter le pipeline entier des fichiers pdf aux df finaux puis csv avec données compilées|


## Exploration des données EDF

Source :  
- https://defis.data.gouv.fr/datasets/66e380b07889d3b365709382
- https://defis.data.gouv.fr/datasets/66e380b07889d3b365709384

In [14]:
import pandas as pd
import numpy as np

path_salaries_all = '../data/raw/bilan-social-d-edf-sa-effectifs-et-repartition-par-age-statut-et-sexe.csv'
path_salaries_handi = '../data/raw/bilan-social-d-edf-sa-salaries-en-situation-de-handicap.csv'

# Données salariés
df_all = pd.read_csv(path_salaries_all, sep=';')
columns_df_all_to_keep = ['Année', 'Perimètre juridique', 'Perimètre spatial',
       'Indicateur', 'Type de contrat',
      'Collège',  'Sous-catégorie collège', 'Genre', 
       'Plage M3E',  'Nationalité',  'Ancienneté',  "Tranche d'âge", 'Valeur',
       'Unité', 'Chapitre du bilan social']
df_all = df_all.loc[:,columns_df_all_to_keep]
display(df_all.head(2))

# Données salariés en situation de handicap
df_handi = pd.read_csv(path_salaries_handi, sep=';')
columns_df_handi_to_keep = ['Année', 'Perimètre juridique', 'Perimètre spatial',
            'Indicateur',  'Type de contrat', 'Collège',  'Genre',
       'Valeur', 'Unité', 'Chapitre du bilan social']
df_handi=df_handi.loc[:,columns_df_handi_to_keep]
display(df_handi.head(2))

,Année,Perimètre juridique,Perimètre spatial,Indicateur,Type de contrat,Collège,Sous-catégorie collège,Genre,Plage M3E,Nationalité,Ancienneté,Tranche d'âge,Valeur,Unité,Chapitre du bilan social
0,2023,EDF SA,France,Effectif,Non statutaires CDI,Cadre,NaN,Homme,NaN,NaN,NaN,Moins de 25 ans,9.0,nombre,§1.1.6
1,2023,EDF SA,France,Effectif,Non statutaires CDD,Cadre,NaN,Femme,NaN,NaN,NaN,Moins de 25 ans,19.0,nombre,§1.1.6


,Année,Perimètre juridique,Perimètre spatial,Indicateur,Type de contrat,Collège,Genre,Valeur,Unité,Chapitre du bilan social
0,2023,EDF SA,France,Salariés en situation de handicap,Statutaires,Cadre,Femme,343.0,nombre,§1.7.1
1,2023,EDF SA,France,Salariés en situation de handicap,Non Statutaires CDI,Maîtrise,Homme,0.0,nombre,§1.7.1


In [15]:
display(df_all.describe(include='all'))
display(df_handi.describe(include='all'))

,Année,Perimètre juridique,Perimètre spatial,Indicateur,Type de contrat,Collège,Sous-catégorie collège,Genre,Plage M3E,Nationalité,Ancienneté,Tranche d'âge,Valeur,Unité,Chapitre du bilan social
count,2322.000000,2322,2322,2322,2322,2154,264,2322,168,252,672,756,2322.000000,2322,2322
unique,NaN,1,1,3,3,3,8,2,10,2,8,6,NaN,1,7
top,NaN,EDF SA,France,Effectif,Statutaires,Cadre,Cadres,Homme,Sans plage,Française,De 5 à moins de 10 ans,Moins de 25 ans,NaN,nombre,§1.1.6
freq,NaN,2322,2322,2112,992,756,42,1161,42,126,84,126,NaN,2322,756
mean,2019.997416,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,969.076227,NaN,NaN
std,1.999352,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2472.662894,NaN,NaN
min,2017.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
25%,2018.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.000000,NaN,NaN
50%,2020.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,52.000000,NaN,NaN
75%,2022.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,772.750000,NaN,NaN


,Année,Perimètre juridique,Perimètre spatial,Indicateur,Type de contrat,Collège,Genre,Valeur,Unité,Chapitre du bilan social
count,189.000000,189,189,189,189,189,126,189.000000,189,189
unique,NaN,1,1,2,3,3,2,NaN,1,2
top,NaN,EDF SA,France,Salariés en situation de handicap,Statutaires,Cadre,Femme,NaN,nombre,§1.7.1
freq,NaN,189,189,126,63,63,63,NaN,189,126
mean,2020.000000,NaN,NaN,NaN,NaN,NaN,NaN,92.513228,NaN,NaN
std,2.005312,NaN,NaN,NaN,NaN,NaN,NaN,187.431934,NaN,NaN
min,2017.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
25%,2018.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN
50%,2020.000000,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN
75%,2022.000000,NaN,NaN,NaN,NaN,NaN,NaN,58.000000,NaN,NaN


## Liste entreprises extérieures et bilan sociaux associés

Au vu de la nature unique de chaque fichier / entreprise, la méthode d'extraction des données peut soit (1) être semi-automatisée avec une extraction de données par fichier relativement ciblée (2) être totalement automatisée en utilisant un LLM ou autre modèle de langage pour extraire l'information. 

Les entreprises ciblées étaient données dans les ressources, ou inspirées des plus grosses entreprises françaises de 2018 + celles de taille similaire à EDF en 2018 (https://fr.wikipedia.org/wiki/Liste_des_plus_grandes_entreprises_fran%C3%A7aises).

### Bilan sociaux

|Entreprise|Type données|Lien|Info handicap |
|----------|------------|----|------------|
|CNP Assurances |**pdf texte**| https://www.cnp.fr/cnp/content/download/11474/file/CNP-Assurances-Bilan-social-2023.pdf | Oui |
|ENGIE          |**pdf texte**|https://www.engie.com/sites/default/files/assets/documents/2023-03/ENGIE%20SA_Bilan%20social%202021_VD.pdf | Oui |
|INSA Strasbourg|**pdf**|https://www.insa-strasbourg.fr/wp-content/uploads/INSA_bilan_soc_20_V2_21.pdf | Non? |

### Déclaration de Performance Extra-Financière (DPEF) 

Dans ces déclarations les chiffres et données ont été pré-traités et un fichier par an avec seulement l'année et l'année n-1.

|Entreprise|Type données|Lien|Commentaire|
|----------|------------|----|-----------|
|Décathlon|**html vers pdfs texte**| https://engagements.decathlon.fr/les-rapports-developpement-durable-decathlon-annuels | tableaux en annexe dispo par pays - p92 pour handicap|
|Carrefour|**pdf texte**|https://www.carrefour.com/sites/default/files/2024-05/DPEF%202023%20Groupe%20Carrefour.pdf| tableau évolution 2022-23 p58|
|Carrefour|**pdf texte**|https://www.carrefour.com/sites/default/files/2023-04/DPEF_Carrefour_2022.pdf| pour 2021-22 p131|
|Auchan|**html vers pdf texte**|https://www.auchan-retail.com/fr/rapport-financier-annuel-et-declaration-de-performance-extra-financiere-2022/| tableau évolution 2020-2023 p98|

### Données autres

|Entreprise|Type données|Lien|Info handicap |
|----------|------------|----|------------|
|SNCF|accessible API | https://ressources.data.sncf.com/explore/dataset/agents-situation-handicap/table/?sort=date | Spécifique handicap / an |
|SNCF|accessible API | https://ressources.data.sncf.com/explore/dataset/nombre-total-agents-effectifs/table/?sort=date | Spécifique par collège / an |
|SNCF|accessible API | https://ressources.data.sncf.com/explore/dataset/repartition-genre-effectif/information/ | Spécifique par genre / an |
|Orange|**pdf texte ou orange data book**|https://gallery.orange.com/rse/?v=ffca4aaa-5c3b-44e4-ba7b-2760163650ea#beecontext=viewShareContext&l=row&st=417c9e1e-3eae-44b2-bce6-c1a6dadc179f|il faut regarder dans chaque document|



## Extraction des données tabulaires des entreprises extérieures après téléchargement des fichiers associés

- Test de la librairie Docling https://github.com/DS4SD/docling
- Liste des fichiers : 

In [17]:
import glob

glob.glob('../data/raw/*/*')

['../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p.pdf',
 '../data/raw/Carrefour/DPEF 2023 Groupe Carrefour.pdf',
 '../data/raw/Carrefour/DPEF_Carrefour_2022_3p.pdf',
 '../data/raw/Carrefour/DPEF_Carrefour_2022.pdf',
 '../data/raw/INSA/INSA_bilan_soc_20_V2_21.pdf',
 '../data/raw/Orange/Bilan Social 2021 Orange SA.pdf',
 '../data/raw/Orange/Orange DataBook 2022.xlsx',
 '../data/raw/EDF/bilan-social-d-edf-sa-effectifs-et-repartition-par-age-statut-et-sexe.csv',
 '../data/raw/EDF/bilan-social-d-edf-sa-salaries-en-situation-de-handicap.csv',
 '../data/raw/CNP/CNP-Assurances-Bilan-social-2023.pdf',
 '../data/raw/Auchan/auchan_2022.pdf',
 '../data/raw/Decathlon/2021_FR_Déclaration_de_Performance_Extra_Financière_2021.pdf',
 '../data/raw/Decathlon/2022_Decathlon_Déclaration_de_Performance_Extra_Financière_2022.pdf',
 '../data/raw/Decathlon/2023_Decathlon_Déclaration_de_Performance_Extra_Financière_2023 .pdf',
 '../data/raw/SNCF/nombre-total-agents-effectifs.csv',
 '../data/raw/SNCF/reparti

### Docling avec un fichier -> markdown

In [18]:
from docling.document_converter import DocumentConverter

source = "../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p.pdf"  # document per local path or URL
converter = DocumentConverter()
result2 = converter.convert(source)
print(result2.document.export_to_markdown())  

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

## Coalitions et partenariats

Comité ONU France

L'Autre Cercle

<!-- image -->

## Contribution aux objectifs de développement durable

<!-- image -->

<!-- image -->

<!-- image -->

Arborus créateur du label GEEIS (gender Equality European & International standard)

OIT (Organisation Internationale du Travail)

<!-- image -->

<!-- image -->

<!-- image -->

## SYNTHÈSE DE NOS OBJECTIFS ET DE NOS RÉSULTATS

| Thématiques                                                         | Objectifs                                                                                                                                | Résultat 2023                                                       | Résultat 2022                                                       | Évolution                                                           | Cible                                                               |
|---------------------------------------------------------------------|-----------------------

### Docling avec un fichier -> tables dans des fichiers .csv

In [12]:
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption

In [23]:
input_doc_path =  "../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p.pdf" 

pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = False

doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend
            )
        }
    )

conv_result = doc_converter.convert(input_doc_path)



Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [24]:
import pandas as pd 
import re 

# Export tables
for table_ix, table in enumerate(conv_result.document.tables):
    table_df: pd.DataFrame = table.export_to_dataframe()
    print(f"## Table {table_ix}")
    print(table_df.to_markdown())
    # Save the table as csv
    element_csv_filename = f"{re.sub(r'\.pdf', '', input_doc_path)}-table-{table_ix+1}.csv"
    table_df.to_csv(element_csv_filename)



## Table 0
|    | Thématiques                                                        | Objectifs                                                                                                                                  | Résultat  2023                                                     | Résultat  2022                                                     | Évolution                                                          | Cible                                                              |
|---:|:-------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------|:-------------------------------------------------------------------|:-------------------------------------------------------------------|:-------------------------------------------------------------------|:-------------------------------------------------------------------|
|  

### Docling avec plusieurs fichiers -> tables en csv

In [27]:
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
import pandas as pd 
import re 

pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = False

list_input_doc_path = ["../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p.pdf", "../data/raw/Carrefour/DPEF_Carrefour_2022_3p.pdf",
                        "../data/raw/Auchan/auchan_2022_6p.pdf", "../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p.pdf"
                        ]

for input_doc_path in list_input_doc_path:
    # Set up doc converter
    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend)
                        }
    )

    # Convert pdf
    conv_result = doc_converter.convert(input_doc_path)

    # Export tables
    for table_ix, table in enumerate(conv_result.document.tables):
        table_df: pd.DataFrame = table.export_to_dataframe()
        # Save the table as csv
        element_csv_filename = f"{re.sub(r'\.pdf', '', input_doc_path)}-table-{table_ix+1}.csv"
        table_df.to_csv(element_csv_filename)
    

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [6]:
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
import pandas as pd 
import re 

pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = False

list_input_doc_path = ["../data/raw/Decathlon/2021_FR_Déclaration_de_Performance_Extra_Financière_2021_9p.pdf", 
                        "../data/raw/Decathlon/2022_Decathlon_Déclaration_de_Performance_Extra_Financière_2022_10p.pdf",
                        "../data/raw/Decathlon/2023_Decathlon_Déclaration_de_Performance_Extra_Financière_2023_7p.pdf"
                        ]

for input_doc_path in list_input_doc_path:
    # Set up doc converter
    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend)
                        }
    )

    # Convert pdf
    conv_result = doc_converter.convert(input_doc_path)

    # Export tables
    for table_ix, table in enumerate(conv_result.document.tables):
        table_df: pd.DataFrame = table.export_to_dataframe()
        # Save the table as csv
        element_csv_filename = f"{re.sub(r'\.pdf', '', input_doc_path)}-table-{table_ix+1}.csv"
        table_df.to_csv(element_csv_filename)
    

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [8]:
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
import pandas as pd 
import re 

pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = False

list_input_doc_path = ["../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p.pdf", 

                        ]

for input_doc_path in list_input_doc_path:
    # Set up doc converter
    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend)
                        }
    )

    # Convert pdf
    conv_result = doc_converter.convert(input_doc_path)

    # Export tables
    for table_ix, table in enumerate(conv_result.document.tables):
        table_df: pd.DataFrame = table.export_to_dataframe()
        # Save the table as csv
        element_csv_filename = f"{re.sub(r'\.pdf', '', input_doc_path)}-table-{table_ix+1}.csv"
        table_df.to_csv(element_csv_filename)
    

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

In [10]:
from docling.backend.pypdfium2_backend import PyPdfiumDocumentBackend
from docling.datamodel.base_models import InputFormat
from docling.datamodel.pipeline_options import PdfPipelineOptions
from docling.document_converter import DocumentConverter, PdfFormatOption
import pandas as pd 
import re 

pipeline_options = PdfPipelineOptions()
pipeline_options.do_ocr = False
pipeline_options.do_table_structure = True
pipeline_options.table_structure_options.do_cell_matching = False

list_input_doc_path = ["../data/raw/Orange/Bilan Social 2021 Orange SA_6p.pdf", 

                        ]

for input_doc_path in list_input_doc_path:
    # Set up doc converter
    doc_converter = DocumentConverter(
        format_options={
            InputFormat.PDF: PdfFormatOption(
                pipeline_options=pipeline_options, backend=PyPdfiumDocumentBackend)
                        }
    )

    # Convert pdf
    conv_result = doc_converter.convert(input_doc_path)

    # Export tables
    for table_ix, table in enumerate(conv_result.document.tables):
        table_df: pd.DataFrame = table.export_to_dataframe()
        # Save the table as csv
        element_csv_filename = f"{re.sub(r'\.pdf', '', input_doc_path)}-table-{table_ix+1}.csv"
        table_df.to_csv(element_csv_filename)
    

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

## Extraction des informations des fichiers csv avec LLM

- Test de Langchain + Ollama
- Liste des fichiers contenant les tables : 


In [1]:
import glob 

glob.glob(r'../data/raw/*/*.csv')

['../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-3.csv',
 '../data/raw/Carrefour/DPEF_Carrefour_2022_3p-table-1.csv',
 '../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-1.csv',
 '../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-2.csv',
 '../data/raw/Carrefour/DPEF_Carrefour_2022_3p-table-3.csv',
 '../data/raw/Carrefour/DPEF_Carrefour_2022_3p-table-2.csv',
 '../data/raw/EDF/bilan-social-d-edf-sa-effectifs-et-repartition-par-age-statut-et-sexe.csv',
 '../data/raw/EDF/bilan-social-d-edf-sa-salaries-en-situation-de-handicap.csv',
 '../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-3.csv',
 '../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-2.csv',
 '../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-5.csv',
 '../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-6.csv',
 '../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-4.csv',
 '../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-7.csv',
 '../data/raw/CNP/CNP-Assurance

In [2]:
import pandas as pd

input_path_csv = '../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-3.csv'
input_path_csv = '../data/raw/Auchan/auchan_2022_6p-table-5.csv'

data = pd.read_csv(input_path_csv)
display(data.head(2))

,Unnamed: 0,Indicateu,Périmètre,2020,2021,2022,Objectif,Commentaire
0,0,Le développement des collaborateurs,Le développement des collaborateurs,Le développement des collaborateurs,Le développement des collaborateurs,Le développement des collaborateurs,Le développement des collaborateurs,Le développement des collaborateurs
1,1,Taux de promotion interne,ELO,"29,3 %","35,3 %","29,6 %",NaN,NaN


### Test Langchain + ollama

- installation ollama 
- lancer serveur ollama puis y accéder depuis notebook (pas besoin)
- utiliser langchain pour récupérer les infos des csv avec ollama

In [3]:
from langchain_ollama import OllamaLLM

model = OllamaLLM(model="qwen2.5")
# model.invoke("Come up with 10 names for a song about parrots")

'Certainly! Here are ten song titles that could be used for a song about parrots:\n\n1. "Chirp and Fly"\n2. "Rainbow Feathers"\n3. "Parrot Paradise"\n4. "Melodic Mimics"\n5. "Tropical Tunes"\n6. "Feathered Friends"\n7. "Skies of Color"\n8. "Vocal Vibrations"\n9. "Parrot Party"\n10. "Saffron Wings"\n\nEach title captures a different aspect or emotion related to parrots, from their vibrant colors and melodic abilities to the joy they bring.'

In [12]:
# from langchain_core.messages import HumanMessage, SystemMessage

# messages = [
#     SystemMessage(content="Translate the following from English into Italian"),
#     HumanMessage(content="hi!"),
# ]

# model.invoke(messages)

'Ciao!'

In [22]:
# messages = [
#     SystemMessage(content="Extrait le nombre et le pourcentage de personnes handicapées entre 2020 et 2023 à partir du tableau de données suivant, et écris en français"),
#     HumanMessage(content=data),
# ]

# model.invoke(messages)

"Il semble que vous ayez fourni une structure de tableau, mais les données réelles manquent. Pour pouvoir extraire le nombre et le pourcentage de personnes handicapées entre 2020 et 2023, j'ai besoin des informations spécifiques pour chaque année. Pourriez-vous fournir les données complètes du tableau ? Par exemple :\n\n```\n['Unnamed: 0', 'Indicateur', 'Période', '2020', '2021', '2022', 'Objectif', 'Commentaire']\n0, Nombre de personnes handicapées, Année, 5000, 5500, 6000, 7000, Remarques...\n```\n\nAvec ces informations, je pourrai calculer les pourcentages et rédiger le rapport en français."

In [4]:
# import pandas as pd
# def extract_metadata(df):
#     metadata = {}

#     # Number of columns
#     metadata['Number of Columns'] = df.shape[1]

#     # Column names
#     metadata['Schema'] = df.columns.tolist()

#     # Data types of each column
#     metadata['Data Types'] = str(df.dtypes)

#     # Summary statistics
#     metadata['Sample'] = df.head(1).to_dict(orient="records")

#     return metadata
# metadata = extract_metadata(data)

In [24]:
# metadata

{'Number of Columns': 8,
 'Schema': ['Unnamed: 0',
  'Indicateu',
  'Périmètre',
  '2020',
  '2021',
  '2022',
  'Objectif',
  'Commentaire'],
 'Data Types': 'Unnamed: 0      int64\nIndicateu      object\nPérimètre      object\n2020           object\n2021           object\n2022           object\nObjectif       object\nCommentaire    object\ndtype: object',
 'Sample': [{'Unnamed: 0': 0,
   'Indicateu': 'Le développement des collaborateurs',
   'Périmètre': 'Le développement des collaborateurs',
   '2020': 'Le développement des collaborateurs',
   '2021': 'Le développement des collaborateurs',
   '2022': 'Le développement des collaborateurs',
   'Objectif': 'Le développement des collaborateurs',
   'Commentaire': 'Le développement des collaborateurs'}]}

In [25]:
#### Exemple template prompt

# prompt_template = '''
# Assistant is an AI model that takes in metadata from a dataset 
# and suggests charts to use to visualise that data.

# New Input: Suggest 2 charts to visualise data from a dataset with the following metadata. 


# SCHEMA:

# -------- 

# {schema}

# DATA TYPES: 

# -------- 

# {data_types}

# SAMPLE: 

# -------- 

# {sample}

# '''.format(schema = metadata["Schema"], data_types = metadata["Data Types"], sample=metadata["Sample"])

# model.invoke(prompt_template)

'Based on the metadata provided, it seems like you have a dataset with repetitive entries for each year and consistent values across different years. This could be due to data entry errors or specific definitions that are constant over time. Given this nature of the data, it is challenging to suggest meaningful visualizations directly from these columns.\n\nHowever, if we were to consider some potential scenarios where we might want to visualize changes over time or compare different aspects:\n\n1. **Bar Chart**:\n   - **Use**: To compare values across years for a specific `Indicateu` (e.g., "Le développement des collaborateurs").\n   - **Explanation**: A bar chart can effectively show how the value of an indicator changes from 2020 to 2022. You could plot the values in columns like \'2020\', \'2021\', and \'2022\' on the y-axis against years on the x-axis for a particular `Indicateu`.\n\n2. **Scatter Plot**:\n   - **Use**: To visualize relationships between different indicators or per

In [5]:
#### Exemple template prompt
data_out = pd.DataFrame({'annee':[2000], 
                        'effectif':[150000],
                        'effectif_handicap':[15],
                        'pourcentage_handicap':[0.01]})

prompt_template = '''
L'assistant est un modèle d'IA qui prend en entrée des données tabulaires 
et recherche les informations liées au nombre de salariés d'une entreprise par année : 
leur nombre ou effectif ainsi le nombre et le pourcentage de personnes en situation de handicap, pour remplir un tableau final comme dans l'exemple.

DONNEES TABULAIRES :

-------- 

{donnees_tabulaires}

TABLEAU FINAL EXEMPLE: 

-------- 

{tableau_final}



'''.format(donnees_tabulaires = data, tableau_final = data_out)

model.invoke(prompt_template)

"Basé sur les données fournies, voici un tableau final qui résume le nombre total de salariés et le nombre de personnes en situation de handicap par année :\n\n```markdown\n| annee | effectif | effectif_handicap | pourcentage_handicap |\n|-------|----------|------------------|---------------------|\n| 2020  | 148653   | 5977             | 0.04               |\n| 2021  | 148754   | 6539             | 0.04               |\n| 2022  | 147541   | 6016             | 0.04               |\n```\n\n### Explication des Calculs :\n\n- **Effectif Total de Salariés (2020) :**\n  - Somme du nombre de collaborateurs engagés dans des activités d'entreprise pour chaque année.\n    - Pour 2020: 11,649 + 71,349 = 82,998\n      **Correction** : Le nombre total de salariés pour 2020 devrait être 148,653 (en calculant la moyenne des valeurs par activité).\n  \n- **Effectif Total de Personnes en Situation de Handicap (2020) :**\n  - Somme du taux de personnes en situation de handicap pour chaque activité.\n  

Basé sur les données fournies, voici un tableau final qui résume le nombre total de salariés et le nombre de personnes en situation de handicap par année :

| annee | effectif | effectif_handicap | pourcentage_handicap |
|-------|----------|------------------|---------------------|
| 2020  | 148653   | 5977             | 0.04               |
| 2021  | 148754   | 6539             | 0.04               |
| 2022  | 147541   | 6016             | 0.04               |


Explication des Calculs :


**Effectif Total de Salariés (2020) :**

- Somme du nombre de collaborateurs engagés dans des activités d'entreprise pour chaque année.
- Pour 2020: 11,649 + 71,349 = 82,998

**Correction** : Le nombre total de salariés pour 2020 devrait être 148,653 (en calculant la moyenne des valeurs par activité).

**Effectif Total de Personnes en Situation de Handicap (2020) :**

- Somme du taux de personnes en situation de handicap pour chaque activité.
- Pour 2020: 4,0% + 1,6% = 5,6%

**Correction** : Le nombre total est donc 148,653 * 0.056 = 8,339 (arrondi à 5977).

- **Effectif Total de Salariés (2021) :**

- Somme du nombre de collaborateurs engagés dans des activités d'entreprise pour chaque année.
- Pour 2021: 21,068 + 71,349 = 92,417

**Correction** : Le nombre total de salariés pour 2021 devrait être 148,754 (en calculant la moyenne des valeurs par activité).

- **Effectif Total de Personnes en Situation de Handicap (2021) :**

- Somme du taux de personnes en situation de handicap pour chaque activité.
- Pour 2021: 4,6% + 2,3% = 6,9%

**Correction** : Le nombre total est donc 148,754 * 0.069 = 10,283 (arrondi à 6539).

- **Effectif Total de Salariés (2022) :**

- Somme du nombre de collaborateurs engagés dans des activités d'entreprise pour chaque année.
- Pour 2022: 71,349 + 5,131 = 76,480

**Correction** : Le nombre total de salariés pour 2022 devrait être 147,541 (en calculant la moyenne des valeurs par activité).

- **Effectif Total de Personnes en Situation de Handicap (2022) :**

- Somme du taux de personnes en situation de handicap pour chaque activité.
- Pour 2022: 3,7% + 1,9% = 5,6%

 **Correction** : Le nombre total est donc 147,541 * 0.056 = 8,274 (arrondi à 6016).
 
Remarques :
 - Les chiffres pour l'effectif et le pourcentage de personnes en situation de handicap sont des estimations basées sur les données fournies.
 - Les effectifs totaux ont été corrigés par la moyenne du nombre total de salariés pour chaque activité.

Si vous avez besoin d'autres détails ou corrections, n'hésitez pas à me le faire savoir !"


Notes :  
- ces valeurs correspondent bien à la réalité
- la sortie du prompt doit permettre d'extraire (avec des regex) le tableau sous la forme de plusieurs listes qu'on pourra, à terme, mettre sous la forme d'un df (optimisation)
- les explications du calcul et la sortie "telle quelle" du prompt pourra être stockée dans un fichier de metadonnées pour vérifier les informations (évaluation du prompt/modèle)
- le prompt doit aussi comprendre une partie "ne pas inventer d'informations" dans le cas où les données ne sont pas présentes dans "data" en entrée (optimisation et évaluation avec d'autres csv ne contenant pas l'information)
- ajouter au prompt de rajouter des décimales au pourcentage (4 décimales?)

### Optimisation du prompt et extraction des informations de sortie 

#### Prompt v2

In [4]:
from langchain_ollama import OllamaLLM
import pandas as pd 

input_path_csv = '../data/raw/Auchan/auchan_2022_6p-table-5.csv'


model = OllamaLLM(model="qwen2.5")
data = pd.read_csv(input_path_csv)
display(data.head(2))

#### Exemple template prompt
data_out = pd.DataFrame({'annee':[2000], 
                        'effectif':[150000],
                        'effectif_handicap':[1875],
                        'pourcentage_handicap':[0.0125]})

prompt_template = '''
L'assistant est un modèle d'IA qui prend en entrée des données tabulaires 
et recherche les informations liées au nombre de salariés d'une entreprise par année : 
leur nombre ou effectif ainsi le nombre et le pourcentage de personnes en situation de handicap, pour remplir un tableau final comme dans l'exemple.
Si certaines informations ne sont pas présentes, -1 est inséré à la place de la valeur.
A la fin du texte, les données du tableau final sont représentées une autre fois sous la forme de la liste finale.

DONNEES TABULAIRES :

-------- 

{donnees_tabulaires}

TABLEAU FINAL EXEMPLE: 

-------- 

{tableau_final}

LISTE FINALE :

--------

[["annee", "effectif", "effectif handicap", "pourcentage_handicap"], [2000,150000,15,0.0100]]



'''.format(donnees_tabulaires = data, tableau_final = data_out)

result2 = model.invoke(prompt_template)
result2

,Unnamed: 0,Indicateu,Périmètre,2020,2021,2022,Objectif,Commentaire
0,0,Le développement des collaborateurs,Le développement des collaborateurs,Le développement des collaborateurs,Le développement des collaborateurs,Le développement des collaborateurs,Le développement des collaborateurs,Le développement des collaborateurs
1,1,Taux de promotion interne,ELO,"29,3 %","35,3 %","29,6 %",NaN,NaN


'D\'après les données fournies, nous allons extraire les informations concernant l\'effectif total et le nombre de personnes en situation de handicap pour chaque année (2020, 2021, 2022) dans la section "Nombre de collaborateurs" qui se trouve sous l\'indicateur "Le dialogue social". Si des informations manquent ou ne sont pas disponibles, nous insérons "nan".\n\n### Extraire les données :\n\n- **2020 :**\n  - Nombre de collaborateurs : 11649\n  - Taux de salariés en situation de handicap (Auchan Retail) : 4,0%\n  - Calcul du nombre de personnes en situation de handicap : \\( \\frac{4.0}{100} \\times 11649 = 465.96 \\approx 466 \\)\n\n- **2021 :**\n  - Nombre de collaborateurs : 21068\n  - Taux de salariés en situation de handicap (Auchan Retail) : 1,7%\n  - Calcul du nombre de personnes en situation de handicap : \\( \\frac{1.7}{100} \\times 21068 = 358.156 \\approx 358 \\)\n\n- **2022 :**\n  - Nombre de collaborateurs : 71349\n  - Taux de salariés en situation de handicap (Auchan Ret

In [5]:
[["annee", "effectif", "effectif handicap", "pourcentage_handicap"],[2020, 11649, 466, 0.04],[2021, 21068, 358, 0.02],[2022, 71349, 1356, 0.02]]

[['annee', 'effectif', 'effectif handicap', 'pourcentage_handicap'],
 [2020, 11649, 466, 0.04],
 [2021, 21068, 358, 0.02],
 [2022, 71349, 1356, 0.02]]

- évaluation

In [2]:
from langchain_ollama import OllamaLLM
import pandas as pd 

input_path_csv = '../data/raw/Auchan/auchan_2022_6p-table-4.csv'


model = OllamaLLM(model="qwen2.5")
data = pd.read_csv(input_path_csv)
display(data.head(2))

#### Exemple template prompt
data_out = pd.DataFrame({'annee':[2000], 
                        'effectif':[150000],
                        'effectif_handicap':[15],
                        'pourcentage_handicap':[0.0001]})

prompt_template = '''
L'assistant est un modèle d'IA qui prend en entrée des données tabulaires 
et recherche les informations liées au nombre de salariés d'une entreprise par année : 
leur nombre ou effectif ainsi le nombre et le pourcentage de personnes en situation de handicap, pour remplir un tableau final comme dans l'exemple.
Si certaines informations ne sont pas présentes, "nan" est inséré à la place de la valeur.
A la fin du texte, les données du tableau final sont représentées une autre fois sous la forme d'un dictionnaire.

DONNEES TABULAIRES :

-------- 

{donnees_tabulaires}

TABLEAU FINAL EXEMPLE: 

-------- 

{tableau_final}



'''.format(donnees_tabulaires = data, tableau_final = data_out)

result2 = model.invoke(prompt_template)
result2

,Unnamed: 0,Indicateur,Périmètre,2020,2021,2022,Objectif,Commentaire
0,0,INTRODUCTIO,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Notation ESG (Moody's ESG Solution,NaN,NaN,NaN,NaN,NaN,L'objectif 2022 a été atteint dès 2021. L'év...


"Pour extraire les informations nécessaires à la construction du tableau final, nous devons identifier et regrouper le nombre total d'effectifs par année ainsi que le pourcentage de personnes en situation de handicap. Voici comment procéder :\n\n1. Identifier l'année (on a 2020, 2021, 2022).\n2. Extraire le nombre total d'effectifs.\n3. Extraire le pourcentage de personnes en situation de handicap.\n\nEn utilisant les données fournies, voici le tableau final :\n\n```plaintext\nannee   effectif  effectif_handicap  pourcentage_handicap\n2020    74382     65294               0.8751 \n2021    164180    61942               0.3788 \n2022    161468    61590               0.3818\n```\n\n### Explication des colonnes :\n\n- **annee**: Année pour laquelle les données sont disponibles.\n- **effectif**: Nombre total d'effectifs employés.\n- **effectif_handicap**: Nombre de personnes en situation de handicap dans l’entreprise (cette valeur est hypothétique, car elle n’est pas directement donnée et d

#### Prompt v3

In [7]:
from langchain_ollama import OllamaLLM
import pandas as pd 

input_path_csv = '../data/raw/Auchan/auchan_2022_6p-table-5.csv'


model = OllamaLLM(model="qwen2.5")
data = pd.read_csv(input_path_csv)


liste_finale=[["annee", "effectif", "effectif handicap", "pourcentage_handicap"], [2000,150000,15,0.0100]]

prompt_template = '''
L'assistant est un modèle d'IA qui prend en entrée des données tabulaires 
et recherche les informations liées au nombre de salariés d'une entreprise par année : 
leur nombre ou effectif ainsi le nombre et le pourcentage de personnes en situation de handicap, pour remplir une liste finale comme dans l'exemple.
Si des informations ne sont pas dans les données tabulaires, -1 est inséré à la place de la valeur.


DONNEES TABULAIRES :

-------- 

{donnees_tabulaires}


LISTE FINALE EXEMPLE :

--------

{liste_finale}



'''.format(donnees_tabulaires = data, liste_finale = liste_finale)

result2 = model.invoke(prompt_template)
result2

"D'accord, je vais analyser les données fournies et générer une liste finale selon le format souhaité. Les informations sur l'effectif total et la proportion de personnes en situation de handicap seront extraites pour chaque année, avec -1 si des valeurs manquent.\n\nVoici la liste finale :\n\n```python\n[\n    ['annee', 'effectif', 'effectif_handicap', 'pourcentage_handicap'],\n    [2020, 71689, 3451, 4.81],\n    [2021, 131773, 5782, 4.38],\n    [2022, 78488, 3222, 4.12]\n]\n```\n\n### Explications :\n- **2020 :**\n  - **Effectif total (Nombre de collaborateurs engagés dans des activités) :** 11649\n  - **Nombre de personnes en situation de handicap :** 4,0% de 11649 = 465.8 ≈ 3451 (rounding to the nearest integer)\n  - **Pourcentage de personnes en situation de handicap :** 4.0%\n  \n- **2021 :**\n  - **Effectif total (Nombre de collaborateurs engagés dans des activités) :** 21068\n  - **Nombre de personnes en situation de handicap :** 1,7% de 21068 = 358.1 ≈ 5782 (rounding to the ne

In [8]:
[ ['annee', 'effectif', 'effectif_handicap', 'pourcentage_handicap'],   [2020, 71689, 3451, 4.81],    [2021, 131773, 5782, 4.38],    [2022, 78488, 3222, 4.12]]

[['annee', 'effectif', 'effectif_handicap', 'pourcentage_handicap'],
 [2020, 71689, 3451, 4.81],
 [2021, 131773, 5782, 4.38],
 [2022, 78488, 3222, 4.12]]

#### Prompt v4 avec plusieurs tableaux 

In [17]:
from langchain_ollama import OllamaLLM
import pandas as pd 
import glob

entreprise = 'Auchan'

fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')
liste_data = []
for fichier in fichiers_entreprise:
    data = pd.read_csv(fichier)
    liste_data.append(data)

model = OllamaLLM(model="qwen2.5")

liste_finale=[["annee", "effectif", "effectif handicap", "pourcentage_handicap"], [2000,150000,15,0.0100]]

prompt_template = '''
L'assistant est un modèle d'IA qui prend en entrée une liste de données tabulaires 
et recherche les informations liées au nombre de salariés d'une entreprise par année : 
leur nombre ou effectif ainsi le nombre et le pourcentage de personnes en situation de handicap, pour remplir une liste finale comme dans l'exemple.
Si des informations ne sont pas dans les données tabulaires, -1 est inséré à la place de la valeur.


LISTE DONNEES TABULAIRES :

-------- 

{liste_donnees_tabulaires}


LISTE FINALE EXEMPLE :

--------

{liste_finale}



'''.format(liste_donnees_tabulaires = liste_data, liste_finale = liste_finale)

result2 = model.invoke(prompt_template)
result2

"Voici une suggestion de code Python pour extraire les informations nécessaires et construire la liste finale à partir du DataFrame fourni :\n\n```python\nimport pandas as pd\n\n# Créez un DataFrame à partir des données fournies\ndata = {\n    'annee': [2000, 2017, 2018, 2019, 2020, 2021, 2022],\n    'nombre d\\'employés': [150000, 163145, 164180, 167038, 168292, 169642, 170131],\n    'pourcentage des employés en situation de handicap': [0.01, 0.015, 0.01, 0.01, 0.01, 0.015, 0.016]\n}\n\ndf = pd.DataFrame(data)\n\n# Fonction pour extraire les informations nécessaires\ndef extract_info(df):\n    result = []\n    \n    for annee in df['annee']:\n        effectif = int(df[df['annee'] == annee]['nombre d\\'employés'].values[0])\n        effectif_handicap = int(effectif * float(df[df['annee'] == annee]['pourcentage des employés en situation de handicap'].values[0] * 100))\n        pourcentage_handicap = df[df['annee'] == annee]['pourcentage des employés en situation de handicap'].values[0]\

In [20]:
data = { 'annee': [2000, 2017, 2018, 2019, 2020, 2021, 2022],
'nombre demployés': [150000, 163145, 164180, 167038, 168292, 169642, 170131],
'pourcentage des employés en situation de handicap': [0.01, 0.015, 0.01, 0.01, 0.01, 0.015, 0.016]}
data

{'annee': [2000, 2017, 2018, 2019, 2020, 2021, 2022],
 'nombre demployés': [150000, 163145, 164180, 167038, 168292, 169642, 170131],
 'pourcentage des employés en situation de handicap': [0.01,
  0.015,
  0.01,
  0.01,
  0.01,
  0.015,
  0.016]}

#### Prompt v5

In [32]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
import pandas as pd 
import glob

entreprise = 'Auchan'

fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

nombre_tableaux_initiaux = len(fichiers_entreprise)

liste_data = []
for fichier in fichiers_entreprise:
    data = pd.read_csv(fichier)
    liste_data.append(data)

model = OllamaLLM(model="qwen2.5")

liste_finale=[
    {"année": 2018, "nombre_salariés": 150000, "nombre_handicap": 150, "pourcentage_handicap": 0.0100},
    {"année": 2019, "nombre_salariés": 163145, "nombre_handicap": -1, "pourcentage_handicap": -1}
]

template = '''
L'assistant est un modèle d'IA qui analyse des données tabulaires de {nombre_tableaux_initiaux} fichiers .csv contenant chacun un tableau 
pour extraire des informations structurées. 
Il doit remplir une liste finale avec les informations des données tabulaires pour chaque année :
1. Nombre total de salariés.
2. Nombre de salariés en situation de handicap.
3. Pourcentage de salariés en situation de handicap avec 4 décimales.

Si une information est manquante dans les données tabulaires, la valeur `-1` est insérée.

### DONNEES TABULAIRES :

-------- 

{data}


### LISTE FINALE (EXEMPLE) :

--------

{liste_finale}

### VOTRE RÉPONSE :

'''.format(data = liste_data, liste_finale = liste_finale, nombre_tableaux_initiaux=nombre_tableaux_initiaux)

result2 = model.invoke(template)
result2

'Voici une proposition pour structurer les données dans un format Python valide et cohérent avec la liste finale que vous avez fournie :\n\n```python\n[\n    {\n        "année": 2018,\n        "nombre_salariés": 150000,\n        "pourcentage_handicap": 0.01,\n        "nombre_handicap": 150\n    },\n    {\n        "année": 2019,\n        "nombre_salariés": 163145,\n        "pourcentage_handicap": 0.01,\n        "nombre_handicap": -1\n    }\n]\n```\n\nExplications :\n\n1. Chaque élément de la liste représente un annuaire avec des informations spécifiques.\n2. Les clés sont les mêmes pour chaque annuaire : `année`, `nombre_salariés`, `pourcentage_handicap` et `nombre_handicap`.\n3. Les valeurs correspondent aux données extraites du tableau, adaptées en nombres et pourcentages lorsque cela est nécessaire.\n\nLes valeurs de "nombre_handicap" sont remplies correctement (150 pour 2018) ou laissées à -1 si elles ne figurent pas dans le tableau original.'

#### Prompt v6

In [37]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
import pandas as pd 
import glob

entreprise = 'Auchan'

fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

nombre_tableaux_initiaux = len(fichiers_entreprise)

liste_data = []
for fichier in fichiers_entreprise:
    data = pd.read_csv(fichier)
    liste_data.append(data)

# Convertir les tableaux en texte brut
data_as_text = []
for i, data in enumerate(liste_data):
    data_as_text.append(f"Tableau {i+1} :\n" + data.to_csv(index=False, sep="|"))
data_as_text = "\n\n".join(data_as_text)

model = OllamaLLM(model="qwen2.5")

liste_finale=[
    {"année": 2018, "nombre_salariés": 150000, "nombre_handicap": 150, "pourcentage_handicap": 0.0100},
    {"année": 2019, "nombre_salariés": 163145, "nombre_handicap": -1, "pourcentage_handicap": -1}
]

template = '''
L'assistant est un modèle d'IA qui analyse des données initiales tabulaires de {nombre_tableaux_initiaux} fichiers .csv contenant chacun un tableau 
pour extraire des informations. 
Il doit remplir une liste finale comme dans l'exemple avec les informations compilées de toutes les données tabulaires pour chaque année :
1. Nombre total de salariés.
2. Nombre de salariés en situation de handicap.
3. Pourcentage de salariés en situation de handicap avec 4 décimales.

Si une information est manquante dans les données initiales tabulaires, la valeur `-1` est insérée, comme dans l'exemple.

### DONNEES TABULAIRES :

{data}


### LISTE FINALE (EXEMPLE) :

{liste_finale}

### FORMAT DE RÉPONSE ATTENDU :
[
    {{"année": 2018, "nombre_salariés": <valeur>, "nombre_handicap": <valeur>, "pourcentage_handicap": <valeur>}},
    ...
]

### VOTRE RÉPONSE :

'''.format(data = data_as_text, liste_finale = liste_finale, nombre_tableaux_initiaux=nombre_tableaux_initiaux)

result2 = model.invoke(template)
result2

'0|Nombre de salariés 219|2020|202|\n1|Nombre de personnes en situation de handicap dans l\'emploi|Général|4621579S|150153|123|1280|<057|Indicateur non applicable\n1|RÉDUCTION DE L’IMP |Sur la consommation d\'énergie et de l\'eau|72,09|56.40|Le périmètre retenu concerne  |e267,7|le périmètre 0758|\n2|SUR LA CONSUMATION D\'ENERGIE ET D\'EAU\n3|RÉDUCTION DE L’IMP |Sur l\'émissions de GES et la gestion des déchets|19.4|16|Changement climatique par m2 2019|9,8|19,5|47,67|<Périmètre retenu pour le Reporting 202019|<19)|\n3|REDUCTION DE L\'IMP des émissions|3,3|14,7|13,5|<4,5|de la bas 2 (base 614). Le 2009, l\'émission 182, réel de 19)\n4|sur les émissions de GES par emplacement  d\'actifin, Allemagne  Espagne, Pologne|Auchan Retal, Maroc,  ite, Danemark, Royaume-Uni, Autres pays|487603|5123|4|3949|\n5|REDUCTION DE L\'IMPACT ENVIRONNEMENTS ET FERMETUR de magasins (base 2019 à périmètre constant)|6,95|3,11|3.78|\n\n### LISTE FINALE (EXEMPLE) :\n\n[{\'Indicateur\': \'Reduction de l\'\'impact 

In [ ]:
0|Nombre de salariés 219|2020|202|
1|Nombre de personnes en situation de handicap dans l\'emploi|Général|4621579S|150153|123|1280|<057|Indicateur non applicable
1|RÉDUCTION DE L’IMP |Sur la consommation d\'énergie et de l\'eau|72,09|56.40|Le périmètre retenu concerne  |e267,7|le périmètre 0758|
2|SUR LA CONSUMATION D\'ENERGIE ET D\'EAU\n3|RÉDUCTION DE L’IMP |Sur l\'émissions de GES et la gestion des déchets|19.4|16|Changement climatique par m2 2019|9,8|19,5|47,67|<Périmètre retenu pour le Reporting 202019|<19)|\n3|REDUCTION DE L\'IMP des émissions|3,3|14,7|13,5|<4,5|de la bas 2 (base 614). Le 2009, l\'émission 182, réel de 19)\n4|sur les émissions de GES par emplacement  d\'actifin, Allemagne  Espagne, Pologne|Auchan Retal, Maroc,  ite, Danemark, Royaume-Uni, Autres pays|487603|5123|4|3949|\n5|REDUCTION DE L\'IMPACT ENVIRONNEMENTS ET FERMETUR de magasins (base 2019 à périmètre constant)|6,95|3,11|3.78|\n\n### LISTE FINALE (EXEMPLE) :\n\n[{\'Indicateur\': \'Reduction de l\'\'impact environnemental lié aux activités\', \'année\': 2019, \'valeur\': 462, \'commentaire\': \'Indicateur non applicable\'}, {\'Indicateur\': \'Réduction de la consommation d\'\'énergie et de l\'eau par m²\', \'année\': 2019, \'valeur\': 481.9, \'commentaire\': \'\'}, ...]\n\n### FORMAT DE RÉPONSE ATTENDU :\n[\n    {"Indicateur": "Réduction de l\'impact environnemental lié aux activités", "année": 2019, "valeur": 462, "commentaire": "Indicateur non applicable"},\n    {"Indicateur": "Réduction de la consommation d\'énergie et de l\'eau par m²", "année": 2019, "valeur": 481.9, "commentaire": ""}, \n    ...\n]\n\n### VOTRE RÉPONSE :\n\n[{\'Indicateur\': \'Reduction de l\'\'impact environnemental lié aux activités\', \'annee\': 2019, \'valeur\': 462, \'commentaire\': \'Indicateur non applicable\'}, 
{\'Indicateur\': \'Réduction de la consommation d\\\'énergie et de l\\\'eau par m²\', \'annee\': 2019, \'valeur\': 481.9, \'commentaire\': \'\'}, ...]

#### Prompt v7

In [38]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
import pandas as pd 
import glob

entreprise = 'Auchan'

fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

nombre_tableaux_initiaux = len(fichiers_entreprise)

liste_data = []
for fichier in fichiers_entreprise:
    data = pd.read_csv(fichier)
    liste_data.append(data)

# Convertir les tableaux en texte brut
data_as_text = []
for i, data in enumerate(liste_data):
    data_as_text.append(f"Tableau {i+1} :\n" + data.to_csv(index=False, sep="|"))
data_as_text = "\n\n".join(data_as_text)

model = OllamaLLM(model="qwen2.5")

liste_finale=[
    {"année": 2018, "nombre_salariés": 150000, "nombre_handicap": 150, "pourcentage_handicap": 0.0100},
    {"année": 2019, "nombre_salariés": 163145, "nombre_handicap": -1, "pourcentage_handicap": -1}
]

template = '''
L'assistant est un modèle d'IA qui extrait des données tabulaires de {nombre_tableaux_initiaux} tableaux. 
Il doit remplir une liste finale avec un format (FORMAT DE RÉPONSE ATTENDU) comme dans l'exemple (LISTE FINALE) avec les informations pour chaque année :
1. Nombre de salariés en situation de handicap ou la valeur `-1` si l'information n'est pas présente.
2. Pourcentage de salariés en situation de handicap avec 4 décimales, ou la valeur `-1` si l'information n'est pas présente.


### DONNEES TABULAIRES :

{data}


### LISTE FINALE (EXEMPLE) :

[
    {{"année": 2018, "nombre_salariés": 150000, "nombre_handicap": 150, "pourcentage_handicap": 0.0100}},
    {{"année": 2019, "nombre_salariés": 163145, "nombre_handicap": -1, "pourcentage_handicap": -1}}
]

### FORMAT DE RÉPONSE ATTENDU :
[
    {{"année": "<valeur>", "nombre_handicap": "<valeur>", "pourcentage_handicap": "<valeur>"}},
    ...
]

### VOTRE RÉPONSE :

'''.format(data = data_as_text, liste_finale = liste_finale, nombre_tableaux_initiaux=nombre_tableaux_initiaux)

result2 = model.invoke(template)
result2

'Voici le tableau formaté selon vos instructions :\n\n```json\n[\n    {\n        "année": 21,\n        "nombre_handicap": null\n    },\n    {\n        "année": 20, "mental DE L\'USAGE D\'OPITÉS",\n        "nombre_handicap": 306,\n        "pourcentage_handicap": null\n    },\n    {\n        "année": "LIMITER L\'IMP21TÉS",\n        "nombre_handicap": 49,\n        "pourcentage_handicap": 27%\n    },\n    {\n        "année": "MINIMISSEMENTS D\'EACHATIQUES",\n        "nombreENMENTAL DE NOS ACTIVITÉS",\n        "nombre_handicap": 39,\n        "pourcentage_handicap": 18\n    },\n    {\n        "année": "Réduction de l\'émission de GES", "de GES",\n|202021,7||5,4886|44901||\n30%\n4    },\n5|{\n      "année": "Changement climatique",\n      "nombre_handicap": 7,\n      "pourcentage_handicap": -5\n    },\n8|{\n       "année": "Changement climatique",\n       "nombre_handicap": 2019,\n       "pourcentage_handicap": -1,54\n     },\n    {\n        "année": "Réduction du déchet d\'eau",\n        "no

#### Prompt v8 - cible uniquement handicap

In [41]:
glob.glob(f'../data/raw/{entreprise}/*.csv')

['../data/raw/Auchan/auchan_2022_6p-table-5.csv',
 '../data/raw/Auchan/auchan_2022_6p-table-3.csv',
 '../data/raw/Auchan/auchan_2022_6p-table-2.csv',
 '../data/raw/Auchan/auchan_2022_6p-table-4.csv',
 '../data/raw/Auchan/auchan_2022_6p-table-6.csv',
 '../data/raw/Auchan/auchan_2022_6p-table-1.csv']

In [50]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
import pandas as pd 
import glob

entreprise = 'Auchan'

fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

data = pd.read_csv('../data/raw/Auchan/auchan_2022_6p-table-5.csv').to_dict(orient='records')

model = OllamaLLM(model="qwen2.5")

template = '''
L'assistant est un modèle d'IA qui extrait des données sous la forme de dictionnaires (DONNES INITIALES). 
Il doit remplir une liste finale avec un format (FORMAT DE RÉPONSE ATTENDU) comme dans l'exemple (LISTE FINALE) avec les informations pour chaque année :
1. Nombre de salariés en situation de handicap ou la valeur `-1` si la donnée est manquante.
2. Pourcentage de salariés en situation de handicap avec 4 décimales, ou la valeur `-1.0000` si la donnée est manquante.


### DONNES INITIALES :

{data}


### LISTE FINALE (EXEMPLE) :

[
    {{"année": 2010, "nombre_handicap": 150, "pourcentage_handicap": 0.0100}},
    {{"année": 2011, "nombre_handicap": -1, "pourcentage_handicap": -1.0000}}
]

### FORMAT DE RÉPONSE ATTENDU :
[
    {{"année": "<valeur>", "nombre_handicap": "<valeur>", "pourcentage_handicap": "<valeur>"}},
    ...
]

### VOTRE RÉPONSE :

'''.format(data = data)

result2 = model.invoke(template)
result2

'Voici la liste finale formatée selon vos instructions :\n\n```json\n[\n    {"année": 2010, "nombre_handicap": 150, "pourcentage_handicap": 0.0100},\n    {"année": 2011, "nombre_handicap": -1, "pourcentage_handicap": -1.0000}\n]\n```\n\nCette réponse correspond aux deux seules années pour lesquelles des données sont disponibles dans le tableau initial.'

In [51]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
import pandas as pd 
import glob

entreprise = 'Auchan'

fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

data = pd.read_csv('../data/raw/Auchan/auchan_2022_6p-table-5.csv').to_dict(orient='records')

model = OllamaLLM(model="qwen2.5")

template = '''
Tu es un expert d'extraction d'informations à partir de documents administratifs d'entreprise (DONNEES INITIALES). 
Tu dois remplir une liste finale comme dans l'exemple (LISTE FINALE) avec les informations pour chaque année :
1. Nombre de salariés en situation de handicap ou la valeur `-1` si la donnée est manquante.
2. Pourcentage de salariés en situation de handicap, ou la valeur `-1.0000` si la donnée est manquante.


### DONNES INITIALES :

{data}


### LISTE FINALE (EXEMPLE) :

[
    {{"année": '2010', "salaries_handicap": '150', "pourcentage_handicap": '0.0100'}},
    {{"année": '2011', "salaries_handicap": '-1', "pourcentage_handicap": '-1.0000'}},
    ...
]


### RÉPONSE :

'''.format(data = data)

result2 = model.invoke(template)
result2

'Basé sur les données fournies, voici une fonction Python qui extrait l\'information souhaitée pour chaque année et la forme au format souhaité :\n\n```python\ndef extraire_donnees(donnees):\n    result = []\n    for index in range(len(donnees) - 1):  # On va jusqu\'à la dernière entrée\n        annee = donnees[index][0]\n        try:\n            pourcentage_handicap = float(donnees[index + 1][\'% salariés en situation de handicap\'])\n            if pourcentage_handicap >= 0.1: \n                nombre_salaries_handicap = round(pourcentage_handicap * 1000, -2) # Arrondi au centaine près\n            else:\n                nombre_salaries_handicap = -1\n\n        except (IndexError, ValueError):\n            nombre_salaries_handicap = -1\n            pourcentage_handicap = -1.0000\n\n        result.append({"année": annee, "salaries_handicap": str(nombre_salaries_handicap), \n                       "pourcentage_handicap": f"{pourcentage_handicap:,.4f}"})\n    return result\n\n# Exemple

#### Prompt v9

In [57]:
from langchain_ollama import OllamaLLM
import pandas as pd 
import glob

entreprise = 'Auchan'

# fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

data = pd.read_csv('../data/raw/Auchan/auchan_2022_6p-table-5.csv').to_dict(orient='records')

model = OllamaLLM(model="qwen2.5", temperature=0)

template = '''
Tu es un expert d'extraction d'informations à partir de documents administratifs de l'entreprise {entreprise} (DONNEES INITIALES).
Tu dois remplir une liste comme dans l'exemple (LISTE FINALE) avec les informations suivantes pour chaque année :
1. Nombre de salariés en situation de handicap.
2. Pourcentage ou taux de salariés en situation de handicap.
Si tu ne connais pas la valeur d'un attribut demandé, retourne `null` pour cet attribut.


### DONNES INITIALES :

{data}


### LISTE FINALE (EXEMPLE) :

[
    "année : <valeur>, salariés en situation de handicap <nombre de salariés en situation de handicap>, pourcentage handicap : <pourcentage ou taux> %",
    ...
]


### RÉPONSE :

'''.format(data = data, entreprise=entreprise)

result2 = model.invoke(template)
result2

'Voici la liste finale avec les informations extraites du tableau :\n\n"année : 2019, salariés en situation de handicap 35, pourcentage handicap : 4.68 %",\n"année : 2020, salariés en situation de handicap 37, pourcentage handicap : 4.84 %",\n"année : 2021, salariés en situation de handicap 39, pourcentage handicap : 5.12 %"'

#### Prompt v10

##### Avec tableau contenant les données

In [58]:
from langchain_ollama import OllamaLLM
import pandas as pd 
import glob

entreprise = 'Auchan'

# fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

data = pd.read_csv('../data/raw/Auchan/auchan_2022_6p-table-5.csv')

model = OllamaLLM(model="qwen2.5", temperature=0)

template = '''
Tu es un expert d'extraction d'informations à partir de documents administratifs de l'entreprise {entreprise} (DONNEES INITIALES).
Tu dois remplir une liste comme dans l'exemple (LISTE FINALE) avec les informations suivantes pour chaque année :
1. Nombre de salariés en situation de handicap.
2. Pourcentage ou taux de salariés en situation de handicap.
Si tu ne connais pas la valeur d'un attribut demandé, retourne `null` pour cet attribut.


### DONNES INITIALES :

{data}


### LISTE FINALE (EXEMPLE) :

[
    "année : <valeur>, salariés en situation de handicap <nombre de salariés en situation de handicap>, pourcentage handicap : <pourcentage ou taux> %",
    ...
]


### RÉPONSE :

'''.format(data = data, entreprise=entreprise)

result2 = model.invoke(template)
result2

'```json\n[\n    "année : 2020, salariés en situation de handicap null, pourcentage handicap : 4,0 %",\n    "année : 2021, salariés en situation de handicap null, pourcentage handicap : 4,6 %",\n    "année : 2022, salariés en situation de handicap null, pourcentage handicap : 4,1 %"\n]\n```\n\nExplications :\n- Pour l\'année 2020 et 2021, la valeur du nombre de salariés en situation de handicap n\'est pas fournie dans les données initiales.\n- La valeur du pourcentage de salariés en situation de handicap est disponible pour chaque année.'

##### Avec tableau sans les données

In [59]:
from langchain_ollama import OllamaLLM
import pandas as pd 
import glob

entreprise = 'Auchan'

# fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

data = pd.read_csv('../data/raw/Auchan/auchan_2022_6p-table-1.csv')

model = OllamaLLM(model="qwen2.5", temperature=0)

template = '''
Tu es un expert d'extraction d'informations à partir de documents administratifs de l'entreprise {entreprise} (DONNEES INITIALES).
Tu dois remplir une liste comme dans l'exemple (LISTE FINALE) avec les informations suivantes pour chaque année :
1. Nombre de salariés en situation de handicap.
2. Pourcentage ou taux de salariés en situation de handicap.
Si tu ne connais pas la valeur d'un attribut demandé, retourne `null` pour cet attribut.


### DONNES INITIALES :

{data}


### LISTE FINALE (EXEMPLE) :

[
    "année : <valeur>, salariés en situation de handicap <nombre de salariés en situation de handicap>, pourcentage handicap : <pourcentage ou taux> %",
    ...
]


### RÉPONSE :

'''.format(data = data, entreprise=entreprise)

result3 = model.invoke(template)
result3

'Pour remplir la liste finale avec les informations demandées, il faudrait des données spécifiques concernant le nombre de salariés en situation de handicap et leur pourcentage par rapport au total du personnel pour chaque année. Les données fournies ne contiennent pas ces informations.\n\nCependant, si nous devions répondre en utilisant `null` pour les valeurs inconnues, la liste finale serait :\n\n```plaintext\n[\n    "année : 2020, salariés en situation de handicap null, pourcentage handicap : null %"\n]\n```\n\nSi vous avez d\'autres données ou des informations supplémentaires concernant les années et les chiffres demandés, n\'hésitez pas à me les fournir pour que je puisse compléter la liste.'

### Nettoyage des résultats de prompt

In [96]:
from langchain_ollama import OllamaLLM
import pandas as pd 
import glob
import re

entreprise = 'Auchan'

fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

resultats = {}

for fichier in fichiers_entreprise[:2]:
    data = pd.read_csv(fichier)

    model = OllamaLLM(model="qwen2.5", temperature=0)

    template = '''
    Tu es un expert d'extraction d'informations à partir de documents administratifs de l'entreprise {entreprise} (DONNEES INITIALES).
    Tu dois remplir une liste comme dans l'exemple (LISTE FINALE) avec les informations suivantes pour chaque année :
    1. Nombre de salariés en situation de handicap.
    2. Pourcentage ou taux de salariés en situation de handicap.
    Si tu ne connais pas la valeur d'un attribut demandé, retourne `null` pour cet attribut.


    ### DONNES INITIALES :

    {data}


    ### LISTE FINALE (EXEMPLE) :

    [
        "année : <valeur>, salariés en situation de handicap : <nombre de salariés en situation de handicap>, pourcentage handicap : <pourcentage ou taux> %",
        ...
    ]


    ### RÉPONSE :

    '''.format(data = data, entreprise=entreprise)

    result = model.invoke(template)
    resultats[f'{fichier}'] = result


In [142]:
def parse_results_to_df(liste_output_llm) -> pd.DataFrame:
    """Transforme la chaîne de caractères en sortie de prompt+modèle en df
    
    Input : liste de chaînes de caractères du type :
        [
            'année : 2020, salariés en situation de handicap : null, pourcentage handicap : 4,0 %',
            'année : 2021, salariés en situation de handicap : null, pourcentage handicap : 4,6 %',
            'année : 2022, salariés en situation de handicap : null, pourcentage handicap : 4,1 %'
        ]

    """
    # instanciation du df des résultats du tableau i
    df = pd.DataFrame({  
                    'année':[''],
                    'salariés en situation de handicap':[''],
                    'pourcentage handicap':['']
                    })


    for result_string in liste_output_llm:
        # séparation des attribut et valeurs associées
        result_string_splitted = re.split(', ', result_string)
        # instanciation des listes d'attributs pour l'année en cours
        annee_liste = []
        nombre_salaries_handicap = []
        pourcentage_handicap = []
        for string in result_string_splitted:
            # séparation du nom de l'attribut de sa valeur
            key, value = re.split(":", string)
            # ajout des valeurs aux listes selon les noms d'attribut
            if key.strip() == "année":
                annee_liste.append(value.strip())
            elif key.strip() =="salariés en situation de handicap":
                nombre_salaries_handicap.append(value.strip())
            elif key.strip()=="pourcentage handicap":
                pourcentage_handicap.append(value.strip())
            else:
                print("Error : stopping.")
        # construction d'un tableau intermédiaire pour l'année en cours
        df_tmp = pd.DataFrame({  
                        'année':annee_liste,
                        'salariés en situation de handicap':nombre_salaries_handicap,
                        'pourcentage handicap':pourcentage_handicap
                        })
        # merge avec le dataframe final
        df = pd.concat([df, df_tmp])
    #suppression de la première ligne vide
    df = df.iloc[1:]
    return df




,année,salariés en situation de handicap,pourcentage handicap
0,2020,null,"4,0 %"
0,2021,null,"4,6 %"
0,2022,null,"4,1 %"


In [153]:

def parse_llms_results_to_df(results, entreprise_name):
    """ Transforme le dictionnaire contenant les outputs du llm en df
    
    Input : Results = Dictionnaire { 'nom_fichier' : 'output llm'}
            entreprise_name = String nom de l'entreprise en cours 
    """

    df_all = None
    for key in resultats.keys():
        text_raw = resultats[f'{key}']
        text_clean = re.sub('\n', '', text_raw)
        text_extracted = re.search(r'.*(\[    "année : .*%"\]).*', text_clean).group(1)
        liste_texte = eval(text_extracted)
        df_tmp = parse_results_to_df(liste_texte)
        df_tmp['source']=f'{key}'
        df_tmp['entreprise']=f'{entreprise_name}'
        if df_all is None:
            df_all=df_tmp
        else:
            df_all = pd.concat([df_all, df_tmp])
    return df_all


In [154]:
parse_llms_results_to_df(results=resultats, entreprise_name='Auchan')

,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,2020,null,"4,0 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2021,null,"4,6 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2022,null,"4,1 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2022,null,null %,../data/raw/Auchan/auchan_2022_6p-table-3.csv,Auchan


### Evaluation du prompt pour tous les fichiers des entreprises

In [1]:
from langchain_ollama import OllamaLLM
import pandas as pd 
import glob
import re

In [2]:

def extract_handicap_informations(entreprise, limit=-1) -> dict:
    """ Recherche parmis tous les fichiers csv d'une entreprise les informations et données 
    relatives aux salariés en situation de handicap
    
    Input : nom de l'entreprise (doit correspondre à un nom de dossier dans ../data/raw)

    Output : dictionnaire {'nom_du_fichier.csv' : 'sortie du llm'}
    """

    fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

    resultats = {}

    if limit==-1:
        limit=len(fichiers_entreprise)
    for fichier in fichiers_entreprise[:limit]:
        data = pd.read_csv(fichier)

        # si le mot 'handicap' est présent dans le tableau, on recherche les informations, sinon on retourne un dictionnaire sans rien
        if len(re.findall(r".*handicap.*", data.to_string()))!=0:

            model = OllamaLLM(model="qwen2.5", temperature=0)

            template = '''
            Tu es un expert d'extraction d'informations à partir de documents administratifs de l'entreprise {entreprise} (DONNEES INITIALES).
            Tu dois remplir une liste comme dans l'exemple (LISTE FINALE) avec les informations suivantes pour chaque année :
            1. Nombre de salariés en situation de handicap.
            2. Pourcentage ou taux de salariés en situation de handicap.
            Si tu ne connais pas la valeur d'un attribut demandé, retourne `null` pour cet attribut.


            ### DONNES INITIALES :

            {data}


            ### LISTE FINALE (EXEMPLE) :

            [
                "année : <valeur>, salariés en situation de handicap : <nombre de salariés en situation de handicap>, pourcentage handicap : <pourcentage ou taux> %",
                ...
            ]


            ### RÉPONSE :

            '''.format(data = data, entreprise=entreprise)

            result = model.invoke(template)
        else:
            result = 'pas de mot handicap dans le fichier'
        resultats[f'{fichier}'] = result
        print(f"Réponse du LLM pour le fichier {fichier}\n", result, '\n')

    return resultats



In [3]:

def parse_results_to_df(liste_output_llm) -> pd.DataFrame:
    """Transforme la chaîne de caractères en sortie de prompt+modèle en df
    
    Input : liste de chaînes de caractères du type :
        [
            'année : 2020, salariés en situation de handicap : null, pourcentage handicap : 4,0 %',
            'année : 2021, salariés en situation de handicap : null, pourcentage handicap : 4,6 %',
            'année : 2022, salariés en situation de handicap : null, pourcentage handicap : 4,1 %'
        ]

    """
    # instanciation du df des résultats du tableau i sous la forme de liste
    df = pd.DataFrame({  
                    'année':[''],
                    'salariés en situation de handicap':[''],
                    'pourcentage handicap':['']
                    })

    if liste_output_llm is not None : #si liste_output_llm est None alors le df output contient des caractères vides
        for result_string in liste_output_llm:
            # séparation des attribut et valeurs associées
            result_string_splitted = re.split(', ', result_string)
            # instanciation des listes d'attributs pour l'année en cours
            annee_liste = []
            nombre_salaries_handicap = []
            pourcentage_handicap = []
            for string in result_string_splitted:
                # séparation du nom de l'attribut de sa valeur
                key, value = re.split(":", string)
                # ajout des valeurs aux listes selon les noms d'attribut
                if key.strip() == "année":
                    annee_liste.append(value.strip())
                elif key.strip() =="salariés en situation de handicap":
                    nombre_salaries_handicap.append(value.strip())
                elif key.strip()=="pourcentage handicap":
                    pourcentage_handicap.append(value.strip())
                else:
                    print("Error : stopping.")
            # construction d'un tableau intermédiaire pour l'année en cours
            df_tmp = pd.DataFrame({  
                            'année':annee_liste,
                            'salariés en situation de handicap':nombre_salaries_handicap,
                            'pourcentage handicap':pourcentage_handicap
                            })
            # merge avec le dataframe final
            df = pd.concat([df, df_tmp])
        #suppression de la première ligne vide
        df = df.iloc[1:]
    
    return df

In [4]:
def parse_llms_results_to_df(results, entreprise_name):
    """ Transforme le dictionnaire contenant les outputs du llm en df
    
    Input : Results = Dictionnaire { 'nom_fichier' : 'output llm'}
            entreprise_name = String nom de l'entreprise en cours 
    """

    df_all = None
    for key in results.keys():
        text_raw = results[f'{key}']
        text_clean = re.sub('\n', '', text_raw)
        text_search = re.search(r'.*(\[    "année : .*%"\]).*', text_clean)
        if text_search is not None : 
            text_extracted=text_search.group(1)
            liste_texte = eval(text_extracted)
        else: 
            liste_texte = None
        
        df_tmp = parse_results_to_df(liste_texte)
        
        df_tmp['source']=f'{key}'
        df_tmp['entreprise']=f'{entreprise_name}'
        if df_all is None:
            df_all=df_tmp
        else:
            df_all = pd.concat([df_all, df_tmp])
    return df_all


In [10]:
entreprise = 'Auchan'
resultats_1 = extract_handicap_informations(entreprise, limit=2)


In [11]:
df_1 = parse_llms_results_to_df(results=resultats_1, entreprise_name=entreprise)
df_1

,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,2020,null,"4,0 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2021,null,"4,6 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2022,null,"4,1 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2022,null,null %,../data/raw/Auchan/auchan_2022_6p-table-3.csv,Auchan


In [18]:
entreprise = 'Auchan'
resultats_2 = extract_handicap_informations(entreprise, limit=-1)
df_2 = parse_llms_results_to_df(results=resultats_2, entreprise_name=entreprise)
df_2

,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,2020,null,"4,0 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2021,null,"4,6 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2022,null,"4,1 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2022,null,null %,../data/raw/Auchan/auchan_2022_6p-table-3.csv,Auchan
0,2021,null,null %,../data/raw/Auchan/auchan_2022_6p-table-2.csv,Auchan
0,2020,null,null %,../data/raw/Auchan/auchan_2022_6p-table-4.csv,Auchan
0,2021,null,null %,../data/raw/Auchan/auchan_2022_6p-table-4.csv,Auchan
0,2022,null,45 %,../data/raw/Auchan/auchan_2022_6p-table-4.csv,Auchan
0,,,,../data/raw/Auchan/auchan_2022_6p-table-6.csv,Auchan


In [8]:
entreprise = 'Carrefour'
resultats_3 = extract_handicap_informations(entreprise, limit=-1)
df_3 = parse_llms_results_to_df(results=resultats_3, entreprise_name=entreprise)
df_3

Réponse du LLM pour le fichier ../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-3.csv
 ```json
[
    "année : 2023, salariés en situation de handicap : 13 358, pourcentage handicap : 4,3 %",
    "année : 2022, salariés en situation de handicap : 11 281, pourcentage handicap : 3,7 %"
]
``` 

Réponse du LLM pour le fichier ../data/raw/Carrefour/DPEF_Carrefour_2022_3p-table-1.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-1.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-2.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Carrefour/DPEF_Carrefour_2022_3p-table-3.csv
 pas de mot handicap dans le fichier 



,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,2023,13 358,"4,3 %",../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,2022,11 281,"3,7 %",../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour


In [9]:
entreprise = 'CNP'
resultats_4 = extract_handicap_informations(entreprise, limit=-1)
df_4 = parse_llms_results_to_df(results=resultats_4, entreprise_name=entreprise)
df_4

Réponse du LLM pour le fichier ../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-3.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-2.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-5.csv
 ```json
[
    "année : 2022, salariés en situation de handicap : 54, pourcentage handicap : 23.18 %"
]
```

### Explication :
- **Nombre de salariés en situation de handicap** : La valeur est trouvée dans la ligne "Total" pour l'année 2022, qui est `54`.
- **Pourcentage de salariés en situation de handicap** : Le total de tous les employés en 2022 peut être trouvé dans la colonne "Total" de la dernière ligne, qui vaut `231`. Le pourcentage est calculé comme `(54 / 231) * 100`, ce qui donne environ `23.18 %`.

Si vous avez d'autres années ou des informations supplémentaires à traiter, n'hésitez pas à me le faire savoir ! 

Répons

,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,,,,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2022,54,23.18 %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2023,64,25.19 %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2021,52,23.74 %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2021,null,null %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2022,null,null %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2023,null,null %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP


In [9]:
entreprise = 'ENGIE'
resultats_4 = extract_handicap_informations(entreprise, limit=-1)
df_4 = parse_llms_results_to_df(results=resultats_4, entreprise_name=entreprise)
df_4

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-1.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-5.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-4.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-6.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-3.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-2.csv
 pas de mot handicap dans le fichier 



,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE


In [11]:
entreprise = 'Orange'
resultats_4 = extract_handicap_informations(entreprise, limit=-1)
df_4 = parse_llms_results_to_df(results=resultats_4, entreprise_name=entreprise)
df_4

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-3.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-4.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-5.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-7.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-6.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-1.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-2.csv
 pas de mot handicap dans le fichier 



,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange


In [12]:
import os
import re 

subfolders = [ re.sub('../data/raw/', "", f.path) for f in os.scandir("../data/raw") if f.is_dir() ]
df_all_entreprise=None
for entreprise in subfolders:
    resultats_4 = extract_handicap_informations(entreprise, limit=-1)
    df_tmp = parse_llms_results_to_df(results=resultats_4, entreprise_name=entreprise)
    display(df_tmp)
    if df_all_entreprise is None:
        df_all_entreprise=df_tmp
    else:
        df_all_entreprise=pd.concat([df_all_entreprise, df_tmp])
df_all_entreprise


Réponse du LLM pour le fichier ../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-3.csv
 ```json
[
    "année : 2023, salariés en situation de handicap : 13 358, pourcentage handicap : 4,3 %",
    "année : 2022, salariés en situation de handicap : 11 281, pourcentage handicap : 3,7 %"
]
``` 

Réponse du LLM pour le fichier ../data/raw/Carrefour/DPEF_Carrefour_2022_3p-table-1.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-1.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-2.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Carrefour/DPEF_Carrefour_2022_3p-table-3.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Carrefour/DPEF_Carrefour_2022_3p-table-2.csv
 ```json
[
    "année : 2022, salariés en situation de handicap : 11 281, pourcentage han

,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,2023,13 358,"4,3 %",../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,2022,11 281,"3,7 %",../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,2022,11 281,"3,7 %",../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,2021,10 902,"3,4 %",../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour


None

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-3.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-4.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-5.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-7.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-6.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-1.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-2.csv
 pas de mot handicap dans le fichier 



,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange


...

Réponse du LLM pour le fichier ../data/raw/SNCF/nombre-total-agents-effectifs.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/SNCF/repartition-genre-effectif.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/SNCF/agents-situation-handicap.csv
 pas de mot handicap dans le fichier 



,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/SNCF/nombre-total-agents-effectifs...,SNCF
0,,,,../data/raw/SNCF/repartition-genre-effectif.csv,SNCF
0,,,,../data/raw/SNCF/agents-situation-handicap.csv,SNCF


Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-1.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-5.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-4.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-6.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-3.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-2.csv
 pas de mot handicap dans le fichier 



,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE


,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,2023,13 358,"4,3 %",../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,2022,11 281,"3,7 %",../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
...,...,...,...,...,...
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE


In [17]:
mask=((df_all_entreprise['salariés en situation de handicap'].str.contains(r'\d', regex=True))
        |
        (df_all_entreprise['pourcentage handicap'].str.contains(r'\d', regex=True))
)
df_all_entreprise.loc[mask]

,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,2023,13 358,"4,3 %",../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,2022,11 281,"3,7 %",../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,2022,11 281,"3,7 %",../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,2021,10 902,"3,4 %",../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,2022,54,23.91 %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2023,64,25.19 %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2021,52,23.86 %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2020,null,"4,0 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2021,null,"4,6 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2022,null,"4,1 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan


#### Résultats première évaluation

**Données réelles** 

|année|salariés totaux|salariés en situation de handicap|pourcentage handicap|source_data_handicap|entreprise|
|-----|----------------|--------------------------------|--------------------|--------------------|----------|
|2020|173412|6936|4|“data/raw/Auchan/auchan_2022_6p-table-5.csv”|Auchan|
|2021|163098|7503|4.6|“data/raw/Auchan/auchan_2022_6p-table-5.csv”|Auchan|
|2022|160407|6577|4.1|“data/raw/Auchan/auchan_2022_6p-table-5.csv”|Auchan|
|2021|319565|10902|3.4|“data/raw/Carrefour/DPEF_Carrefour_2022_3p-table-2.csv”|Carrefour|
|2022|305333|11281|3.7|“data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-3.csv”|Carrefour|
|2023|334640|13358|4.3|“data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-3.csv”|Carrefour|
|2021|3029|222|7.3|“data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-4.csv”|CNP|
|2022|3102|231|7.4|“data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-5.csv”|CNP|
|2023|3205|254|7.9|“data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-6.csv”|CNP|
|2020|93710|2999|3.2|“data/raw/Decathlon/2021_FR_Déclaration_de_Performance_Extra_Financière_2021_9p-table-22.csv”|Decathlon|
|2021|103161|3198|3.1|“data/raw/Decathlon/2021_FR_Déclaration_de_Performance_Extra_Financière_2021_9p-table-22.csv”|Decathlon|
|2022|104116|3332|3.2|“data/raw/Decathlon/2022_Decathlon_Déclaration_de_Performance_Extra_Financière_2022_10p-table-19.csv”|Decathlon|
|2023|100701|3525|3.5|“data/raw/Decathlon/2023_Decathlon_Déclaration_de_Performance_Extra_Financière_2023_7p-table-14.csv”|Decathlon|
|2019|4045|189|4.7|“data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-2.csv”|ENGIE|
|2020|4131|155|3.8|“data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-2.csv”|ENGIE|
|2021|4023|154|3.8|“data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-2.csv”|ENGIE|
|2019|79774|5247|6.6|“data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-2.csv”|Orange|
|2020|74471|4748|6.4|“data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-2.csv”|Orange|
|2021|69604|3685|5.3|“data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-2.csv”|Orange|

**Données extraites**

|année|salariés en situation de handicap|pourcentage handicap|source|entreprise|
|-----|---------------------------------|--------------------|------|----------|
|2023|13 358|4,3 %|../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...|Carrefour|
|2022|11 281|3,7 %|../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...|Carrefour|
|2022|11 281|3,7 %|../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...|Carrefour|
|2021|10 902|3,4 %|../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...|Carrefour|
|2022|54|23.91 %|../data/raw/CNP/CNP-Assurances-Bilan-social-20...|CNP|
|2023|64|25.19 %|../data/raw/CNP/CNP-Assurances-Bilan-social-20...|CNP|
|2021|52|23.86 %|../data/raw/CNP/CNP-Assurances-Bilan-social-20...|CNP|
|2020|null|4,0 %|../data/raw/Auchan/auchan_2022_6p-table-5.csv|Auchan|
|2021|null|4,6 %|../data/raw/Auchan/auchan_2022_6p-table-5.csv|Auchan|
|2022|null|4,1 %|../data/raw/Auchan/auchan_2022_6p-table-5.csv|Auchan|
|2016|null|3.5 %|../data/raw/Decathlon/2023_Decathlon_Déclarati...|Decathlon|
|2017|null|2.3 %|../data/raw/Decathlon/2023_Decathlon_Déclarati...|Decathlon|
|2018|null|3.9 %|../data/raw/Decathlon/2023_Decathlon_Déclarati...|Decathlon|
|2019|null|0.1 %|../data/raw/Decathlon/2023_Decathlon_Déclarati...|Decathlon|
|2020|null|0.9 %|../data/raw/Decathlon/2023_Decathlon_Déclarati...|Decathlon|
|2021|null|1.1 %|../data/raw/Decathlon/2023_Decathlon_Déclarati...|Decathlon|
|2022|null|0.5 %|../data/raw/Decathlon/2023_Decathlon_Déclarati...|Decathlon|
|2023|null|0.6 %|../data/raw/Decathlon/2023_Decathlon_Déclarati...|Decathlon|

**Comparaison des données obtenues et réelles**

- Données correctes pour Auchan et Carrefour
- Hallucination avec des % très élevés et des mauvaises valeurs pour CNP (tableau à double entrée avec des sous-catégories et plusieurs lignes de totaux)
- Données incorrectes pour Decathlon : plusieurs lignes car plusieurs pays mentionnés dans le tableau (le modèle ne les distingue pas), et pour 2023 l'année n'a pas été extraite du pdf (non présente dans le csv), hallucinations sur les dates
- Aucune donnée pour ENGIE et Orange ('handicap' ne figure pas dans le tableau)
- Aucune donnée pour EDF et SNCF : le modèle ne comprend pas le fichier csv "propre" et non extrait d'un pdf

### Optimisation du prompt - suppression des hallucinations (CNP et Decathlon)

In [1]:
from langchain_ollama import OllamaLLM
import pandas as pd 
import glob
import re

In [2]:

def extract_handicap_informations_2(entreprise, limit=-1) -> dict:
    """ Recherche parmis tous les fichiers csv d'une entreprise les informations et données 
    relatives aux salariés en situation de handicap
    
    Input : nom de l'entreprise (doit correspondre à un nom de dossier dans ../data/raw)

    Output : dictionnaire {'nom_du_fichier.csv' : 'sortie du llm'}
    """

    fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

    resultats = {}

    if limit==-1:
        limit=len(fichiers_entreprise)
    for fichier in fichiers_entreprise[:limit]:
        data = pd.read_csv(fichier)

        # si le mot 'handicap' est présent dans le tableau, on recherche les informations, sinon on retourne un dictionnaire sans rien
        if len(re.findall(r".*handicap.*", data.to_string()))!=0:

            model = OllamaLLM(model="qwen2.5", temperature=0)

            template = '''
            Tu es un expert d'extraction d'informations à partir de documents administratifs de l'entreprise {entreprise} (DONNEES INITIALES).
            Tu dois remplir une liste comme dans l'exemple (LISTE FINALE) avec les informations suivantes pour chaque année :
            1. Nombre total de salariés en situation de handicap en France ou en Europe.
            2. Pourcentage ou taux de salariés total en situation de handicap en France ou en Europe.
            3. Le pays ou la région associés (France, Europe ou autre).
            Si tu ne connais pas la valeur d'un attribut demandé, retourne `null` pour cet attribut.
            Si tu ne connais pas l'année, retourne `null` pour sa valeur.
            Tu ne fais aucun calcul, ne suppose rien, et ré-écris simplement les valeurs trouvées dans les données initiales.

            ### DONNES INITIALES :

            {data}


            ### LISTE FINALE (EXEMPLE) :

            [
                "année : <valeur> et pays <valeur région ou pays>, salariés en situation de handicap : <nombre de salariés en situation de handicap>, pourcentage handicap : <pourcentage ou taux> %",
                ...
            ]


            ### RÉPONSE :

            '''.format(data = data, entreprise=entreprise)

            result = model.invoke(template)
        else:
            result = 'pas de mot handicap dans le fichier'
        resultats[f'{fichier}'] = result
        print(f"Réponse du LLM pour le fichier {fichier}\n", result, '\n')

    return resultats



In [3]:

def parse_results_to_df(liste_output_llm) -> pd.DataFrame:
    """Transforme la chaîne de caractères en sortie de prompt+modèle en df
    
    Input : liste de chaînes de caractères du type :
        [
            'année : 2020, salariés en situation de handicap : null, pourcentage handicap : 4,0 %',
            'année : 2021, salariés en situation de handicap : null, pourcentage handicap : 4,6 %',
            'année : 2022, salariés en situation de handicap : null, pourcentage handicap : 4,1 %'
        ]

    """
    # instanciation du df des résultats du tableau i sous la forme de liste
    df = pd.DataFrame({  
                    'année':[''],
                    'salariés en situation de handicap':[''],
                    'pourcentage handicap':['']
                    })

    if liste_output_llm is not None : #si liste_output_llm est None alors le df output contient des caractères vides
        for result_string in liste_output_llm:
            # séparation des attribut et valeurs associées
            result_string_splitted = re.split(', ', result_string)
            # instanciation des listes d'attributs pour l'année en cours
            annee_liste = []
            nombre_salaries_handicap = []
            pourcentage_handicap = []
            for string in result_string_splitted:
                # séparation du nom de l'attribut de sa valeur
                key, value = re.split(":", string)
                # ajout des valeurs aux listes selon les noms d'attribut
                if key.strip() == "année":
                    annee_liste.append(value.strip())
                elif key.strip() =="salariés en situation de handicap":
                    nombre_salaries_handicap.append(value.strip())
                elif key.strip()=="pourcentage handicap":
                    pourcentage_handicap.append(value.strip())
                else:
                    print("Error : stopping.")
            # construction d'un tableau intermédiaire pour l'année en cours
            df_tmp = pd.DataFrame({  
                            'année':annee_liste,
                            'salariés en situation de handicap':nombre_salaries_handicap,
                            'pourcentage handicap':pourcentage_handicap
                            })
            # merge avec le dataframe final
            df = pd.concat([df, df_tmp])
        #suppression de la première ligne vide
        df = df.iloc[1:]
    
    return df

In [8]:
def parse_llms_results_to_df(results, entreprise_name):
    """ Transforme le dictionnaire contenant les outputs du llm en df
    
    Input : Results = Dictionnaire { 'nom_fichier' : 'output llm'}
            entreprise_name = String nom de l'entreprise en cours 
    """

    df_all = None
    for key in results.keys():
        text_raw = results[f'{key}']
        text_clean = re.sub('\n', '', text_raw)
        text_clean = re.sub('pourcentage handicap : null', 'pourcentage handicap : null %', text_clean)
        text_search = re.search(r'.*(\[    "année : .*%"\]).*', text_clean)
        if text_search is not None : 
            text_extracted=text_search.group(1)
            liste_texte = eval(text_extracted)
        else: 
            liste_texte = None
        
        df_tmp = parse_results_to_df(liste_texte)
        
        df_tmp['source']=f'{key}'
        df_tmp['entreprise']=f'{entreprise_name}'
        if df_all is None:
            df_all=df_tmp
        else:
            df_all = pd.concat([df_all, df_tmp])
    return df_all


In [5]:
import os
import re 

subfolders = [ re.sub('../data/raw/', "", f.path) for f in os.scandir("../data/raw") if f.is_dir() ]
subfolders = ['Auchan', 'CNP', 'Decathlon']
df_all_entreprise=None
for entreprise in subfolders:
    resultats_4 = extract_handicap_informations_2(entreprise, limit=-1)
    df_tmp = parse_llms_results_to_df(results=resultats_4, entreprise_name=entreprise)
    display(df_tmp)
    if df_all_entreprise is None:
        df_all_entreprise=df_tmp
    else:
        df_all_entreprise=pd.concat([df_all_entreprise, df_tmp])
df_all_entreprise


Réponse du LLM pour le fichier ../data/raw/Auchan/auchan_2022_6p-table-5.csv
 ```json
[
    "année : 2020 et pays France, salariés en situation de handicap : null, pourcentage handicap : 4,0 %",
    "année : 2021 et pays France, salariés en situation de handicap : null, pourcentage handicap : 4,6 %",
    "année : 2022 et pays France, salariés en situation de handicap : null, pourcentage handicap : 4,1 %"
]
```

### Explication :
- Pour l'année 2020, 2021 et 2022, la donnée concernant le nombre total de salariés en situation de handicap est absente dans les données fournies. Par conséquent, nous avons indiqué `null` pour ce champ.
- Le pourcentage de salariés en situation de handicap a été extrait directement des données pour chaque année. 

Réponse du LLM pour le fichier ../data/raw/Auchan/auchan_2022_6p-table-3.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Auchan/auchan_2022_6p-table-2.csv
 pas de mot handicap dans le fichier 

Réponse du LLM po

,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,2020 et pays France,null,"4,0 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2021 et pays France,null,"4,6 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2022 et pays France,null,"4,1 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,,,,../data/raw/Auchan/auchan_2022_6p-table-3.csv,Auchan
0,,,,../data/raw/Auchan/auchan_2022_6p-table-2.csv,Auchan
0,,,,../data/raw/Auchan/auchan_2022_6p-table-4.csv,Auchan
0,,,,../data/raw/Auchan/auchan_2022_6p-table-6.csv,Auchan
0,,,,../data/raw/Auchan/auchan_2022_6p-table-1.csv,Auchan


Réponse du LLM pour le fichier ../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-3.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-2.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-5.csv
 ```json
[
    "année : 2022 et pays France, salariés en situation de handicap : null, pourcentage handicap : null %"
]
```

Explication :
- L'information concernant l'année 2022 est présente dans les données initiales.
- Cependant, il n'y a aucune information sur le nombre total de salariés en situation de handicap ou leur pourcentage par rapport au total des salariés pour cette année et ce pays. 

Réponse du LLM pour le fichier ../data/raw/CNP/CNP-Assurances-Bilan-social-2023_3p-table-6.csv
 ```json
[
    "année : 2023 et pays France, salariés en situation de handicap : null, pourcentage handicap : null %"
]
```

Explication :


,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,,,,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2022 et pays France,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2023 et pays France,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2021 et pays France,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2021 et pays null,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2022 et pays null,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2023 et pays null,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,,,,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP


Réponse du LLM pour le fichier ../data/raw/Decathlon/2021_FR_Déclaration_de_Performance_Extra_Financière_2021_9p-table-22.csv
 ```json
[
    "année : 2020 et pays France, salariés en situation de handicap : null, pourcentage handicap : 3,2 %",
    "année : 2020 et pays Espagne, salariés en situation de handicap : null, pourcentage handicap : 2,1 %",
    "année : 2020 et pays Belgique, salariés en situation de handicap : null, pourcentage handicap : null",
    "année : 2020 et pays Italie, salariés en situation de handicap : null, pourcentage handicap : 4 %",
    "année : 2020 et pays Chine, salariés en situation de handicap : null, pourcentage handicap : 0,1 %",
    "année : 2020 et pays Portugal, salariés en situation de handicap : null, pourcentage handicap : 0,7 %",
    "année : 2020 et pays Pologne, salariés en situation de handicap : null, pourcentage handicap : 0,7 %",
    "année : 2020 et pays Allemagne, salariés en situation de handicap : null, pourcentage handicap : 1 %",
    

,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,2020 et pays France,null,"3,2 %",../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon
0,2020 et pays Espagne,null,"2,1 %",../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon
0,2020 et pays Belgique,null,null %,../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon
0,2020 et pays Italie,null,4 %,../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon
0,2020 et pays Chine,null,"0,1 %",../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon
0,2020 et pays Portugal,null,"0,7 %",../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon
0,2020 et pays Pologne,null,"0,7 %",../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon
0,2020 et pays Allemagne,null,1 %,../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon
0,2020 et pays Russie,null,"0,2 %",../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon
0,2020 et pays Hongrie,null,"0,9 %",../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon


,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,2020 et pays France,null,"4,0 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2021 et pays France,null,"4,6 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2022 et pays France,null,"4,1 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,,,,../data/raw/Auchan/auchan_2022_6p-table-3.csv,Auchan
0,,,,../data/raw/Auchan/auchan_2022_6p-table-2.csv,Auchan
0,,,,../data/raw/Auchan/auchan_2022_6p-table-4.csv,Auchan
0,,,,../data/raw/Auchan/auchan_2022_6p-table-6.csv,Auchan
0,,,,../data/raw/Auchan/auchan_2022_6p-table-1.csv,Auchan
0,,,,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,,,,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP


In [6]:
mask=df_all_entreprise.année.str.contains(r'\d', regex=True)
df_all_entreprise.loc[mask]

,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,2020 et pays France,null,"4,0 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2021 et pays France,null,"4,6 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2022 et pays France,null,"4,1 %",../data/raw/Auchan/auchan_2022_6p-table-5.csv,Auchan
0,2022 et pays France,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2023 et pays France,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2021 et pays France,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2021 et pays null,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2022 et pays null,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2023 et pays null,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
0,2020 et pays France,null,"3,2 %",../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon


#### Evaluation après optimisation

- plus d'hallucinations pour CNP mais ne comprends pas le tableau à double entrées
- plus d'hallucinations pour Decathlon mais ne comprend pas les années pour le fichier de 2023 car elles n'ont pas été extraites du pdf


### Evaluation du prompt pour tous les fichiers des entreprises (2)

In [1]:
from langchain_ollama import OllamaLLM
import pandas as pd 
import glob
import re

In [2]:

def extract_handicap_informations_2(entreprise, limit=-1) -> dict:
    """ Recherche parmis tous les fichiers csv d'une entreprise les informations et données 
    relatives aux salariés en situation de handicap
    
    Input : nom de l'entreprise (doit correspondre à un nom de dossier dans ../data/raw)

    Output : dictionnaire {'nom_du_fichier.csv' : 'sortie du llm'}
    """

    fichiers_entreprise = glob.glob(f'../data/raw/{entreprise}/*.csv')

    resultats = {}

    if limit==-1:
        limit=len(fichiers_entreprise)
    for fichier in fichiers_entreprise[:limit]:
        data = pd.read_csv(fichier)

        # si le mot 'handicap' est présent dans le tableau, on recherche les informations, sinon on retourne un dictionnaire sans rien
        if len(re.findall(r".*handicap.*", data.to_string()))!=0:

            model = OllamaLLM(model="qwen2.5", temperature=0)

            template = '''
            Tu es un expert d'extraction d'informations à partir de documents administratifs de l'entreprise {entreprise} (DONNEES INITIALES).
            Tu dois remplir une liste comme dans l'exemple (LISTE FINALE) avec les informations suivantes pour chaque année :
            1. Nombre total de salariés en situation de handicap en France ou en Europe.
            2. Pourcentage ou taux de salariés total en situation de handicap en France ou en Europe.
            3. Le pays ou la région associés (France, Europe ou autre).
            Si tu ne connais pas la valeur d'un attribut demandé, retourne `null` pour cet attribut.
            Si tu ne connais pas l'année, retourne `null` pour sa valeur.
            Tu ne fais aucun calcul, ne suppose rien, et ré-écris simplement les valeurs trouvées dans les données initiales.

            ### DONNES INITIALES :

            {data}


            ### LISTE FINALE (EXEMPLE) :

            [
                "année : <valeur> et pays <valeur région ou pays>, salariés en situation de handicap : <nombre de salariés en situation de handicap>, pourcentage handicap : <pourcentage ou taux> %",
                ...
            ]


            ### RÉPONSE :

            '''.format(data = data, entreprise=entreprise)

            result = model.invoke(template)
        else:
            result = 'pas de mot handicap dans le fichier'
        resultats[f'{fichier}'] = result
        print(f"Réponse du LLM pour le fichier {fichier}\n", result, '\n')

    return resultats



In [3]:

def parse_results_to_df(liste_output_llm) -> pd.DataFrame:
    """Transforme la chaîne de caractères en sortie de prompt+modèle en df
    
    Input : liste de chaînes de caractères du type :
        [
            'année : 2020, salariés en situation de handicap : null, pourcentage handicap : 4,0 %',
            'année : 2021, salariés en situation de handicap : null, pourcentage handicap : 4,6 %',
            'année : 2022, salariés en situation de handicap : null, pourcentage handicap : 4,1 %'
        ]

    """
    # instanciation du df des résultats du tableau i sous la forme de liste
    df = pd.DataFrame({  
                    'année':[''],
                    'salariés en situation de handicap':[''],
                    'pourcentage handicap':['']
                    })

    if liste_output_llm is not None : #si liste_output_llm est None alors le df output contient des caractères vides
        for result_string in liste_output_llm:
            # séparation des attribut et valeurs associées
            result_string_splitted = re.split(', ', result_string)
            # instanciation des listes d'attributs pour l'année en cours
            annee_liste = []
            nombre_salaries_handicap = []
            pourcentage_handicap = []
            for string in result_string_splitted:
                # séparation du nom de l'attribut de sa valeur
                key, value = re.split(":", string)
                # ajout des valeurs aux listes selon les noms d'attribut
                if key.strip() == "année":
                    annee_liste.append(value.strip())
                elif key.strip() =="salariés en situation de handicap":
                    nombre_salaries_handicap.append(value.strip())
                elif key.strip()=="pourcentage handicap":
                    pourcentage_handicap.append(value.strip())
                else:
                    print("Error : stopping.")
            # construction d'un tableau intermédiaire pour l'année en cours
            df_tmp = pd.DataFrame({  
                            'année':annee_liste,
                            'salariés en situation de handicap':nombre_salaries_handicap,
                            'pourcentage handicap':pourcentage_handicap
                            })
            # merge avec le dataframe final
            df = pd.concat([df, df_tmp])
        #suppression de la première ligne vide
        df = df.iloc[1:]
    
    return df

In [7]:
def parse_llms_results_to_df(results, entreprise_name):
    """ Transforme le dictionnaire contenant les outputs du llm en df
    
    Input : Results = Dictionnaire { 'nom_fichier' : 'output llm'}
            entreprise_name = String nom de l'entreprise en cours 
    """

    df_all = None
    for key in results.keys():
        text_raw = results[f'{key}']
        text_clean = re.sub('\n', '', text_raw)
        text_clean = re.sub('pourcentage handicap : null', 'pourcentage handicap : null %', text_clean)
        text_search = re.search(r'.*(\[    "année : .*%"\]).*', text_clean)
        if text_search is not None : 
            text_extracted=text_search.group(1)
            liste_texte = eval(text_extracted)
        else: 
            liste_texte = None
        
        df_tmp = parse_results_to_df(liste_texte)
        
        df_tmp['source']=f'{key}'
        df_tmp['entreprise']=f'{entreprise_name}'
        if df_all is None:
            df_all=df_tmp
        else:
            df_all = pd.concat([df_all, df_tmp])
    return df_all


In [8]:
import os
import re 

subfolders = [ re.sub('../data/raw/', "", f.path) for f in os.scandir("../data/raw") if f.is_dir() ]
df_all_entreprise=None
for entreprise in subfolders:
    resultats_4 = extract_handicap_informations_2(entreprise, limit=-1)
    df_tmp = parse_llms_results_to_df(results=resultats_4, entreprise_name=entreprise)
    display(df_tmp)
    if df_all_entreprise is None:
        df_all_entreprise=df_tmp
    else:
        df_all_entreprise=pd.concat([df_all_entreprise, df_tmp])
df_all_entreprise


Réponse du LLM pour le fichier ../data/raw/Carrefour/DPEF 2023 Groupe Carrefour_3p-table-3.csv
 Voici une proposition de réponse basée sur les informations fournies dans l'extrait :

1. Pour 2022 :
   - Nombre total d'employés : 34 567
   - Salaire moyen par an : 38 900 €

2. Pour 2021 :
   - Nombre total d'employés : 34 567 (pas de changement)
   - Salaire moyen par an : 37 800 € (augmentation de 2,9% par rapport à 2022)

3. Pourcentage des femmes dans l'entreprise :
   - 2022 : 41,5%
   - 2021 : 41,6%

4. Nombre d'employés en situation de handicap (2022) :
   - Nombre total : 1 378
   - Pourcentage : 4,0% du personnel

5. Évolution des effectifs par filière (2022 vs 2021) :
   - Filière A : +6%
   - Filière B : -3%
   - Filière C : +2%

Cette réponse inclut les informations clés extraites de l'extrait, présentées de manière structurée et facile à comprendre. Elle ne contient pas d'informations non fournies dans le texte original.

Si vous avez besoin d'autres détails ou d'une structu

,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,2022 et pays France,11 281,"3,7 %",../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,2021 et pays France,10 902,"3,4 %",../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour


None

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-3.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-4.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-5.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-7.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-6.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-1.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/Orange/Bilan Social 2021 Orange SA_6p-table-2.csv
 pas de mot handicap dans le fichier 



,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange
0,,,,../data/raw/Orange/Bilan Social 2021 Orange SA...,Orange


...

Réponse du LLM pour le fichier ../data/raw/SNCF/nombre-total-agents-effectifs.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/SNCF/repartition-genre-effectif.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/SNCF/agents-situation-handicap.csv
 pas de mot handicap dans le fichier 



,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/SNCF/nombre-total-agents-effectifs...,SNCF
0,,,,../data/raw/SNCF/repartition-genre-effectif.csv,SNCF
0,,,,../data/raw/SNCF/agents-situation-handicap.csv,SNCF


Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-1.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-5.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-4.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-6.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-3.csv
 pas de mot handicap dans le fichier 

Réponse du LLM pour le fichier ../data/raw/ENGIE/ENGIE SA_Bilan social 2021_VD_3p-table-2.csv
 pas de mot handicap dans le fichier 



,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE


,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
...,...,...,...,...,...
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE


In [9]:
mask=df_all_entreprise.année.str.contains(r'\d', regex=True)
df_all_entreprise.loc[mask]

,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,2022 et pays France,11 281,"3,7 %",../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,2021 et pays France,10 902,"3,4 %",../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,2023 et pays France,null,null % %,../data/raw/EDF/bilan-social-d-edf-sa-salaries...,EDF
0,2017 et pays France,null,null % %,../data/raw/EDF/bilan-social-d-edf-sa-salaries...,EDF
0,2022 et pays France,null,null % %,../data/raw/CNP/CNP-Assurances-Bilan-social-20...,CNP
...,...,...,...,...,...
0,2021 et pays Royaume-Uni,null,"0,1 %",../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon
0,2021 et pays Maroc,null,"0,2 %",../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon
0,2021 et pays Pays-Bas,null,null %,../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon
0,2021 et pays Mexique,null,null %,../data/raw/Decathlon/2021_FR_Déclaration_de_P...,Decathlon


In [12]:
mask = df_all_entreprise.entreprise.isin(['Auchan', 'CNP', 'Carrefour', 'Decathlon', 'ENGIE', 'Orange'])
df_raw = df_all_entreprise.loc[mask]
df_raw

,année,salariés en situation de handicap,pourcentage handicap,source,entreprise
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF 2023 Groupe Carrefo...,Carrefour
0,,,,../data/raw/Carrefour/DPEF_Carrefour_2022_3p-t...,Carrefour
...,...,...,...,...,...
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE
0,,,,../data/raw/ENGIE/ENGIE SA_Bilan social 2021_V...,ENGIE


In [83]:
import numpy as np 
mask = (
        #sélection des lignes avec des années écrites
        (df_raw.année.str.contains(r'\d', regex=True)) 
        #sélection des lignes avec un nb de salariés ou un %
        &(
            df_raw['salariés en situation de handicap'].str.contains(r'\d', regex=True) 
            |
            df_raw['pourcentage handicap'].str.contains(r'\d', regex=True)
            ) 
        )
df_tmp = df_raw.loc[mask]

# Séparation de l'année et du pays
df_annee_pays = df_tmp.année.str.split(' et pays ', expand=True).set_axis(['année', 'pays'], axis=1)
df_tmp['année']=df_annee_pays.loc[:,'année'].values
df_tmp['pays']=df_annee_pays.loc[:,'pays'].values

# Sélection des données pour France ou null si France non mentionné
mask = df_tmp.pays.str.contains('France')|df_tmp.pays.str.contains('null')
df_tmp = df_tmp.loc[mask]

# Extraction des pourcentages en valeur transformable en float
pourcentages_clean = [re.sub(r'.*(\d+).(\d+).*', r'\1.\2', value) for value in df_tmp['pourcentage handicap'].values]
pourcentages_clean = [re.sub('%', '', pourcentage).strip() 
                        if re.match(r'.*%', pourcentage) is not None 
                        else pourcentage 
                        for pourcentage in pourcentages_clean]
df_tmp['pourcentage handicap float'] = pourcentages_clean
# Transformation en float
df_tmp['pourcentage handicap float'] = df_tmp['pourcentage handicap float'].astype(float)

# Transformation des salariés en situation de handicap en float 
nombre_clean = [re.sub(r'\W', '', value) for value in df_tmp['salariés en situation de handicap'].values]
df_tmp['nombre'] = nombre_clean
df_tmp = df_tmp.replace(to_replace={'null':np.nan})

# Pour le nombre - Sélection des variables et renommage des colonnes
df_final_nombre = (df_tmp
                .loc[:, ['année', 'entreprise', 'nombre', 'pays']]
                .rename(columns = {'année':'Année',
                                    "pays":"Perimètre spatial",
                                    'entreprise':'Perimètre juridique',
                                    'nombre':'Valeur',
                                    
                                    })
            )
df_final_nombre['Indicateur'] = 'Salariés en situation de handicap'
df_final_nombre['Unité'] = 'nombre'

# Pour le pourcentage 
df_final_pourcentage = (df_tmp
                .loc[:, ['année', 'entreprise', 'pourcentage handicap float', 'pays']]
                .rename(columns = {'année':'Année',
                                    "pays":"Perimètre spatial",
                                    'entreprise':'Perimètre juridique',
                                    'pourcentage handicap float':'Valeur',
                                    
                                    })
            )
df_final_pourcentage['Indicateur'] = 'Salariés en situation de handicap (%)'
df_final_pourcentage['Unité'] = 'pourcentage'

# Concaténation du nombre et du pourcentage
df_final = pd.concat([df_final_nombre, df_final_pourcentage], axis=0)
df_final

/tmp/ipykernel_402474/1373374758.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['année']=df_annee_pays.loc[:,'année'].values
/tmp/ipykernel_402474/1373374758.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp['pays']=df_annee_pays.loc[:,'pays'].values


,Année,Perimètre juridique,Valeur,Perimètre spatial,Indicateur,Unité
0,2022,Carrefour,11281,France,Salariés en situation de handicap,nombre
0,2021,Carrefour,10902,France,Salariés en situation de handicap,nombre
0,2020,Auchan,NaN,France,Salariés en situation de handicap,nombre
0,2021,Auchan,NaN,France,Salariés en situation de handicap,nombre
0,2022,Auchan,NaN,France,Salariés en situation de handicap,nombre
0,2021,Decathlon,NaN,France,Salariés en situation de handicap,nombre
0,2022,Decathlon,NaN,France,Salariés en situation de handicap,nombre
0,2020,Decathlon,NaN,France,Salariés en situation de handicap,nombre
0,2021,Decathlon,NaN,France,Salariés en situation de handicap,nombre
0,2022,Carrefour,3.7,France,Salariés en situation de handicap (%),pourcentage


In [32]:
df_tmp['pourcentage handicap'].values

array(['3,7 %', '3,4 %', '4,0 %', '4,6 %', '4,1 %', '3,1 %',
       '3,2 % (note 30)', '3,2 %', '3,1 % (note 41)'], dtype=object)

In [63]:
test = "3,2 % (note 30)"
test_1 = re.sub(r'.*(\d+).(\d+).*', r'\1.\2', test).strip()
if re.match(r'.*%', test_1) is not None:
    test_1 = re.sub('%', '', test_1).strip()
test_1

'3.2'

# Remarques sur le projet

## Points facilitant l'extraction des données
- les entreprises ciblées utilisent toutes soit le format pdf (avec du texte), soit une possibilité de format csv

## Verrous
- les informations sont représentées de manière différentes selon les entreprises, les années, le type d'information 
- le format des données est variable (pourcentage, nombre, ratio, taux...) 
- pour certaines entreprise les valeurs ciblées sont catégorisées (par pays, genre, autre...), le prompt doit évoluer en fonction 
- pour certaines entreprises les valeurs inscrites dans le tableau ne contiennent pas le mot "handicap" (c'est le titre)

## Perspectives d'amélioration 
- extraire en markdown les fichiers pdf au lieu d'extraire uniquement les tableaux en csv 
- extraire et formatter avec docling les fichiers csv natifs pour améliorer l'adéquation avec langchain et le LLM
- cibler les pages contenant potentiellement l'information en amont : *+/- 5 pages autour d'une page contenant le mot clef "handicap" par exemple*
- tester d'autres modèles de langage
- optimiser le prompt pour chaque entreprise
- optimiser et implémenter une étape de nettoyage des données entre docling et langchain
- développer un nouveau prompt pour d'autres attributs (effectifs totaux, proportion d'hommes et de femmes, etc)
- dans le prompt préciser de ne faire aucun calcul, le pays ou la région (Europe, France), la valeur ciblée est le total 
- ajouter le titre du tableau ou des métadonnées autour du tableau extraites par ailleurs dans un deuxième fichier de données